In [1]:
import pandas as pd
import os
import pydicom as dicom
import cv2
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import sys
import morphsnakes as ms
from skimage import morphology
from skimage.measure import label
from skimage.segmentation import active_contour
from skimage import data, io, img_as_ubyte,filters
import matplotlib as mpl
%matplotlib inline
mpl.rc('image', interpolation='none')
plt.rcParams['figure.figsize'] = (7.0, 7.0)

from astropy.table import QTable #for calculation tables
from tabulate import tabulate #for calculation tables

# VOLUME

## Get Images

In [2]:
#vol!!
def getimages (image):
    Ivol=sitk.GetArrayFromImage(image[:, :, :]) #get all slices
    Ivol8= np.zeros([Ivol.shape[0], Ivol.shape[1], Ivol.shape[2]], dtype='uint8') 
    for i in range(Ivol.shape[0]):
        Ivol8[i]=cv2.normalize(src=Ivol[i], dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    j=Ivol8.shape[0]
    plotx=3
    return Ivol8,j,plotx

# Ivol8,j,plotx=getimages(image)
# stackimages(Ivol8)

In [3]:
def stackimages(image, x=0, y=0):
    fig, axs = plt.subplots ((j//plotx)+1, plotx, figsize=(17,65))
    for i in range(j):
        axs[y, x].imshow(image[i], cmap='bone', vmin=0, vmax=255)
        axs[y, x].set_title(f"File {files} slice {i+1}", fontsize=12) #note added "files"
        if x <(plotx-1):
            x+=1
        else:
            x=0
            y+=1

def compare(im1, im2):  
    axs[i,0].imshow(im1[i], 'bone')
    axs[i,1].imshow(im2[i], 'bone')

### Inhomogeneity Correction

#### For Subcutaneous Fat

In [4]:
#functions for inhomogeneity correction - same name as for single slice but added slice + file number
def correct_subcfat(image,mask):
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    output = corrector.Execute( inputImage, mask)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #need to normalize, not direct conversion by "np.uint8"

    return image_c

def correct_roi(image):
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    corrector = sitk.N4BiasFieldCorrectionImageFilter()
    output = corrector.Execute( inputImage)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #need to normalize, not direct conversion by "np.uint8"
    return image_c

#only difference between the two is in the use of a mask for the corrector

In [5]:
def multi_otsu(image): #only difference from multi_otsu_slice is title (includes slice # here)
    motsuth=filters.threshold_multiotsu(image, classes=3)
    #print(motsuth)
    #print (f"Slice {i+1} otsu threshold={motsuth[1]}")
    
    regions=np.digitize(image,bins=motsuth)
    output=img_as_ubyte(regions)
    

#     fig, ax=plt.subplots (1,2, figsize=(10,5))
#     ax[0].imshow(image, cmap="bone")
#     ax[0].set_title(f"File {files} Slice {i+1} Original") #note added "files" #this differs for single participant analysis
#     ax[0].axis("off")

    #ax[1].hist(image.ravel(),bins=255)

    #for th in motsuth:
            #ax[1].axvline(th,color="r")
    #ax[2].imshow(regions,cmap="Accent")
    #ax[2].set_title("Multi-Otsu Result")
    #ax[2].axis ("off")
#     ax[1].imshow(image>motsuth[1],cmap="bone")
#     ax[1].set_title("Threshold Applied")
#     ax[1].axis ("off")
    return motsuth[1] #(fat+ muscle th)

In [6]:
def IH_mask(image): #function to get mask to be used in IH correction
    IH_th=multi_otsu(image) #apply multiotsu to get subcutaneous fat threshold
    IH_mask=label(image>IH_th)
    #IH_mask= (morphology.remove_small_objects(IH_mask,min_size=1000, connectivity=1)) #should we remove holes too or nah??
    IH_mask= cv2.normalize(src=IH_mask, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    ret, IH_mask = cv2.threshold(IH_mask,0,1,cv2.THRESH_BINARY) 
    #   plt.imshow(IH_mask)
    IH_mask=sitk.GetImageFromArray(IH_mask) #convert to sitk image
    return IH_mask

In [7]:
def get_IH1vol_mask(): #get mask from uncorrected image Ivol8
    #Get subfat mask to use for inhomogoneity correction
    #need this mask or else shadow on top right won't be corrected and prevent proper creation of subcfat mask
    IH1vol_mask=[] 
    for i in range(j):
        IH1vol_mask.append([])
        IH1vol_mask[i]=IH_mask(Ivol8[i])
    #convert from image to array to view masks
#     show_IH1vol_mask= np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
#     for i in range(j):
#       show_IH1vol_mask[i]=sitk.GetArrayFromImage(IH1vol_mask[i]) 
#     stackimages(show_IH1vol_mask)
    return IH1vol_mask


#Apply inhomogeneity correction with subcfat masks on Ivol8 slices
def IH1():
    preIvol8c_roi= np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    for i in range(j):
        preIvol8c_roi[i]=correct_subcfat(Ivol8[i],IH1vol_mask[i])
#         fig, axs = plt.subplots (1, 2, figsize=(10,8))
#         axs[0].imshow(Ivol8[i], cmap='bone')
#         axs[0].set_title(f"Slice {i+1} Before Correction") #took out slice # cus error in IH1 function
#         axs[1].imshow(preIvol8c_roi[i], cmap='bone')
#         axs[1].set_title(f"Slice {i+1} After Correction")
    return preIvol8c_roi

In [8]:
# IH1vol_mask=get_IH1vol_mask()
# preIvol8c_roi=IH1()

In [9]:
#From corrected images get another subcfat mask for each slice - generates better masks to be used for correction

def get_IH2vol_mask():
    IH2vol_mask=[] 
    for i in range(j):
        IH2vol_mask.append([])
        IH2vol_mask[i]=IH_mask(preIvol8c_roi[i])
    return IH2vol_mask

#Apply inhomogeneity correction new (better) subcfat mask to Ivol8c slices
def IH2():
    IH3vol= np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    for i in range(j):  
        IH3vol[i]=correct_subcfat(Ivol8[i],IH2vol_mask[i]) #change name
#         fig, axs = plt.subplots (1, 2, figsize=(10,8))
#         axs[0].imshow(Ivol8[i], cmap='bone')
#         axs[0].set_title(f"Slice {i+1} Before Correction") #took out slice # cus error in IH1 function
#         axs[1].imshow(IH3vol[i], cmap='bone')
#         axs[1].set_title(f"Slice {i+1} After Correction")
    return IH3vol


In [10]:
# IH2vol_mask=get_IH2vol_mask()
# IH3vol=IH2()

#### For ROI

In [11]:
#Apply inhomogeneity correction without mask - need for proper correction of roi area (too bright if use mask to correct)
#use these images to obtain roi later


def IH_roi():
    Ivol8c_roi= np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

    for i in range(j):
        Ivol8c_roi[i]=correct_roi(Ivol8[i])
#         fig, axs = plt.subplots (1, 2, figsize=(10,8))
#         axs[0].imshow(Ivol8[i], cmap='bone')
#         axs[0].set_title(f"Slice {i+1} Before Correction")
#         axs[1].imshow(Ivol8c_roi[i], cmap='bone')
#         axs[1].set_title(f"Slice {i+1} After Correction")
    return Ivol8c_roi
# Ivol8c_roi=IH_roi()

## Muscle Mask

In [12]:
#subcfat = threshold2(thigh mask - muscle mask)

In [13]:
def subcfat(image, x=0, y=0): 
    
    fatotsvol=subcfatvol=presubcfatvol=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    subcfatvol_th=[]

    for i in range(j):
        subcfatvol_th.append([])
        subcfatvol_th[i]=multi_otsu(image[i])
        fatotsvol[i]=label(image[i]>subcfatvol_th[i]) #changed this from OG code - got same np.mean value
        
        #subcfatvol[i] = morphology.remove_small_objects(presubcfatvol[i],min_size=600, connectivity=1)
        subcfatvol[i] = (morphology.remove_small_holes(fatotsvol[i],area_threshold=16, connectivity=1)).astype(int) #or min_size (gets warning)
        subcfatvol[i]=cv2.normalize(src=subcfatvol[i], dst=None, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
           
       
    return subcfatvol
            
# IH3vol_mask=subcfat(IH3vol) #change name - IH2_mask indicated mask TO BE USED IH2 correction
# stackimages(IH3vol_mask) 

In [14]:
#Apply inhomogeneity correction new (better) subcfat mask to Ivol8c slices

def IH3():
    Ivol8c_subcfat= np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    k=[]
    for i in range(j):
        k.append([])
        k[i]=sitk.GetImageFromArray(IH3vol_mask[i]) #??why need this when it's already sitk

    for i in range(j):  
        Ivol8c_subcfat[i]=correct_subcfat(Ivol8[i],k[i])
#         fig, axs = plt.subplots (1, 2, figsize=(10,8))
#         axs[0].imshow(Ivol8[i], cmap='bone')
#         axs[0].set_title(f"Slice {i+1} Before Correction")
#         axs[1].imshow(Ivol8c_subcfat[i], cmap='bone')
#         axs[1].set_title(f"Slice {i+1} After Correction")
    return Ivol8c_subcfat
# Ivol8c_subcfat=IH3()

In [15]:
#apply filter

def CurvatureFlowImageFilter(image):
    inputImage=sitk.GetImageFromArray(image)
    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32 )
    corrector = sitk.CurvatureFlowImageFilter()
    corrector.SetNumberOfIterations( 15 );
    corrector.SetTimeStep( 0.1 )
    output = corrector.Execute( inputImage)
    image_c= sitk.GetArrayFromImage(output)
    image_c=cv2.normalize(src=image_c, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #need to normalize, not direct conversion by "np.uint8"
    return image_c

def apply_CurvatureFilter():
    curvature_flow=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
#     fig, axs= plt.subplots(j, 2, figsize=(15, 10))  #why did he have j+1
    for i in range(j):
        curvature_flow[i]=CurvatureFlowImageFilter(Ivol8c_subcfat[i])
#         axs[i,0].imshow(Ivol8c_subcfat[i], 'bone')
#         axs[i,1].imshow(curvature_flow[i], 'bone')
        
    return curvature_flow
# curvature_flow=apply_CurvatureFilter()

In [16]:
#try the snake with just enhanced sharpness; need to convert from im to array with PIL
#next, could try to set a floor value so that there is decreased competition from the contour of the muscle
#make an experimentation file and a separate file for teh master copy

def enhance_sharpness():
    from PIL import ImageEnhance
    from PIL import Image

    Ivol8c_c_s=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        f=curvature_flow[i].copy()
        f=Image.fromarray(f, mode=None)
        enhancer = ImageEnhance.Sharpness(f)
        factor =  6.0
        Ivol8c_c_s[i]=enhancer.enhance(factor)
    return Ivol8c_c_s


def get_subcfatvol():   #Get motsuth[1] threshold for Ivol8c_c_s?? to make subcfat
    subcfatvol=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        subcfatvol[i]=Ivol8c_c_s[i]>multi_otsu(Ivol8c_c_s[i])  #multi_otsu(Ivol8c_c_s[i]) returns threshold (motsuth[1]) -deleted OG code, same np.mean
#         fig, ax=plt.subplots (1,2, figsize=(15, j*15))
#         ax[0].imshow(Ivol8c_c_s[i], cmap="bone")
#         ax[0].set_title(f"Slice {i+1} Original")
#         ax[0].axis("off")
#         ax[1].imshow(subcfatvol[i],cmap="bone")
#         ax[1].set_title("Threshold Applied")
#         ax[1].axis ("off")
        
        #subcfatvol[i] = (morphology.remove_small_holes(subcfatvol[i],area_threshold=40, connectivity=1)) #took out extra for loop from OG code
    
    return subcfatvol



# subcfatvol=get_subcfatvol()
# stackimages(subcfatvol)

 

In [17]:
#TESTING FILTER FOR FASCIA ENHANCEMENT

def BilateralFilter(image):
    bilateral_filter =np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
#     fig, axs= plt.subplots(j, 2, figsize=(15, j*15))  #*why j*15
    for i in range(j):
        bilateral_filter[i]=cv2.bilateralFilter(image[i],20,35,35)
        bilateral_filter[i]=cv2.normalize(src=bilateral_filter[i], dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
#         axs[i,0].imshow(Ivol8c_roi[i], 'bone')
#         axs[i,1].imshow(bilateral_filter[i], 'bone')
        
    return bilateral_filter

# bilateral_filter=BilateralFilter()


In [18]:
def enlarge_size(image): #added!!
    image_big =np.zeros([Ivol8c_roi.shape[0], 712, 712], dtype='uint8') #try to change this later  - unlike def reduce_size need this, cus vstack and hstack work in one 2D?
    for i in range(j):
        top_bottom=np.zeros([100, 512], dtype='uint8')    #y,x
        top_added=np.vstack((top_bottom,image[i])) #first im on top #total shape(y,x) =(256+40=296,256)
        bottom_added=np.vstack((top_added,top_bottom)) #note we adding ONTO top_added NOT Ivol8c[i] #total shape(y,x)=(296+40=336=,256)
        right_left=np.zeros([712, 100], dtype='uint8')
        right_added=np.hstack((bottom_added,right_left))
        left_added=np.hstack((right_left,right_added))
        image_big[i]=left_added.copy()
    return image_big

def reduce_size(image_big): #added
    top_removed=np.delete(image_big, np.s_[:100], 1)  #start from beginning along y axis, take away 40 (top to down) #shape (y,x)=(336-40=296,336)
    bottom_removed=np.delete(top_removed, np.s_[512:], 1) #start at 256 to the end (=40) . shape(y,x)= (296-40=256,336) #note we removing FROM top_removed NOT image_big[i]
    left_removed=np.delete(bottom_removed, np.s_[:100], 2) #start left side take 40 along x-axis (left to right), shape (y,x)=(256,336-40=296)
    right_removed=np.delete(left_removed, np.s_[512:], 2)  #start at 256 to the end (=40) . shape(y,x)= (256,296-40=256)
    image_reg=right_removed.copy()
    return image_reg



In [19]:
def thigh_th(image): #only difference from multi_otsu_slice is title (includes slice # here)
    motsuth=filters.threshold_multiotsu(image, classes=3)
    regions=np.digitize(image,bins=motsuth)
    output=img_as_ubyte(regions)
    return motsuth[0] #(fat+ muscle th)

def get_thighpremask(image):
  IH_mask =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
  for i in range(j):      
      IH_th=thigh_th(image[i]) #apply multiotsu to get subcutaneous fat threshold
      IH_mask[i]=label(image[i]>IH_th)
      IH_mask[i]= cv2.normalize(src=IH_mask[i], dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
      ret, IH_mask[i] = cv2.threshold(IH_mask[i],0,1,cv2.THRESH_BINARY) 

  return IH_mask
#


#
def thigh_mask():
    thighmask_holes =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    skin_fat=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    eroded=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    t=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        im_ff=thighpremask[i].copy()
        h, w = im_ff.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        cv2.floodFill(im_ff, mask, (0,0), 1);
        th, thighmask_holes[i] = cv2.threshold(im_ff, 0, 1, cv2.THRESH_BINARY_INV)
    
    thighmask = thighmask_holes|thighpremask
    thighmask=enlarge_size(thighmask)
    for i in range(j):
        thighmask[i] = (morphology.remove_small_objects(label(thighmask[i]),min_size=8000, connectivity=1))
        th, thighmask[i] = cv2.threshold(thighmask[i], 0, 1, cv2.THRESH_BINARY)
        kernel = np.ones((51,51),np.uint8)
        thighmask[i] = cv2.morphologyEx(thighmask[i], cv2.MORPH_CLOSE, kernel)
        thighmask[i] = (morphology.remove_small_holes(thighmask[i],area_threshold=8000, connectivity=1))
    thighmask=reduce_size(thighmask)
    for i in range(j):
        t[i] = np.uint8(morphology.remove_small_objects(label(thighmask[i]),min_size=8000, connectivity=1))
        eroded[i]=cv2.erode(t[i],np.ones((8,8),np.uint8),iterations = 1)
        
    skin_fat=t-eroded
    return thighmask, skin_fat


#stackimages(subcfatvol_augmented)



t=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
for i in range(j):
    t[i] = (morphology.remove_small_objects(label(thighmask[i]),min_size=8000, connectivity=1))
    
eroded=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    eroded[i]=cv2.erode(t[i],np.ones((8,8),np.uint8),iterations = 1)

skin_fat=t-eroded

newmask=t*(IH3vol_mask+skin_fat)
stackimages(newmask)

In [20]:
def floodfill(im):
    
    premusclemaskvol=musclemaskvol=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    im_floodfillvol=im.copy()
         
    h, w = im.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    kernel = np.ones((11,11),np.uint8)
    musclemaskvol = cv2.morphologyEx(im_floodfillvol, cv2.MORPH_CLOSE, kernel)

    cv2.floodFill(musclemaskvol, mask, (0,0), 1);

    th, musclemaskvol = cv2.threshold(musclemaskvol, 0, 1, cv2.THRESH_BINARY_INV)




    return musclemaskvol

In [21]:
#everything within the subcfat (skin is excluded)

def thigh_minus_skin():
    no_skin=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        kernel = np.ones((15,15),np.uint8)
        no_skin[i]= cv2.morphologyEx(subcfatvol_augmented[i], cv2.MORPH_CLOSE, kernel)
        no_skin[i] = (morphology.remove_small_holes(label(no_skin[i]),area_threshold=5000, connectivity=1))
        filler=floodfill(no_skin[i])
        no_skin[i]=no_skin[i]|filler
    return no_skin
# no_skin=thigh_minus_skin()
# stackimages(no_skin)


In [22]:
def darkpieces():
    t=np.uint8(no_skin-subcfatvol_augmented)
    
   
    t2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    t22=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    for i in range(j):

        t2[i] = (morphology.remove_small_objects(label(t[i]),min_size=10, connectivity=1))
        #t2[i]=cv2.morphologyEx(t2[i], cv2.MORPH_CLOSE, np.ones((2,2),np.uint8))
        t2[i]= cv2.dilate(t2[i],np.ones((3,3),np.uint8),iterations = 1)
        t2[i] = (morphology.remove_small_objects((t2[i]).astype(bool),min_size=170, connectivity=1))
        t2[i]= cv2.erode(t2[i],np.ones((3,3),np.uint8),iterations = 1)

        th, t2[i]= cv2.threshold(np.uint8(t2[i]), 0, 1, cv2.THRESH_BINARY)
        
        t2[i]=cv2.morphologyEx(t2[i], cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
        t2[i] = (morphology.remove_small_objects(label(t2[i]).astype(bool),min_size=1700, connectivity=1))
        th, t2[i]= cv2.threshold(np.uint8(t2[i]), 0, 1, cv2.THRESH_BINARY)
        t2[i] = (morphology.remove_small_holes(label(t2[i]).astype(bool),area_threshold=8000, connectivity=1))
        t2[i]=cv2.morphologyEx(t2[i], cv2.MORPH_OPEN, np.ones((15,15),np.uint8))
        t2[i] = (morphology.remove_small_objects(label(t2[i]).astype(bool),min_size=1700, connectivity=1))
    
        for i in range(j):
            t22[i]=cv2.morphologyEx(t2[i], cv2.MORPH_OPEN, np.ones((31,31),np.uint8))
            t22[i] = (morphology.remove_small_objects(label(t22[i]).astype(bool),min_size=1700, connectivity=1))
            th, t22[i]= cv2.threshold(np.uint8(t22[i]), 0, 1, cv2.THRESH_BINARY)
        
        test=t2-t22
        test[test==255]=0
        test2=t-test
        test2[test2==255]=0
        t=test2
    
    t[t==255] = 0
    
    return t, t2

# t, t2=darkpieces()
# stackimages(t)


In [23]:
def check(im,to_overlay,y):
    overlay = np.ma.masked_where(to_overlay == 0, to_overlay)
    axs[y,1].imshow(im, cmap="bone")
    axs[y,1].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.5)
    axs[y, 1].set_title(f"slice {i+1}", fontsize=12)

#fig, axs = plt.subplots(j, 2, figsize = (12, 200)) #need this??
#for i in range(j):
#   axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')
#   check(Ivol8c_roi[i], t2[i], i)


In [24]:
#this will also remove small dots inside ROI, but will be corrected later...

def remove_vessels(image):
    
    otsvol=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    subcfatvol=presubcfatvol=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    otsu_th=[]

    for i in range(j):
        otsu_th.append([])
        otsu_th[i]=multi_otsu(image[i])

        ret, otsvol[i] = cv2.threshold(image[i],otsu_th[i],1,cv2.THRESH_BINARY)
        presubcfatvol[i] = label(otsvol[i])
        subcfatvol[i] = (morphology.remove_small_holes(presubcfatvol[i],area_threshold=90, connectivity=1)).astype(int) #or min_size (gets warning)
        subcfatvol[i]=cv2.normalize(src=subcfatvol[i], dst=None, alpha=0.0, beta=1.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            
        if i==(j-1):

            return otsu_th, subcfatvol, otsvol
            
#subcfatvol=subcfat(Ivol8c_subcfat) #why comment ?? need??
#stackimages(subcfatvol)#why comment ?? need??


# otsu, vessels_removed_mask, vessels_present=remove_vessels(Ivol8c_c_s)
# stackimages(vessels_removed_mask)
    

In [25]:
#draws the holes that were removed (those that are below teh island threshold)

# pre_vessels_mask=vessels_removed_mask-vessels_present
# stackimages(pre_vessels_mask)

In [26]:
#floodfill objects in a 3d image

def floodfillall(image):
    floodfilled=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        im_floodfill = image[i].copy()
        h, w = im_floodfill.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        cv2.floodFill(im_floodfill, mask, (0,0), 255);
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        floodfilled[i] = image[i] | im_floodfill_inv
        th, floodfilled[i] = cv2.threshold(floodfilled[i], 0, 1, cv2.THRESH_BINARY)
    
    return floodfilled

In [27]:
#area check- function of radius and length to isolate the circles (represents vessels)


def get_vessels():
    contoursvol=[]
    hiercvol=[]
    contour_listvol=[]
    boneROIvol=[]
    bonemwvol=[]

    r=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        contoursvol.append([])
        hiercvol.append([])
        contour_listvol.append([])
        boneROIvol.append([])
        bonemwvol.append([])

    for i in range (j):
        a, b =  cv2.findContours(pre_vessels_mask[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contoursvol[i].append(a)
        hiercvol[i].append(b)

        for contour in contoursvol[i][0]:
            approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)

            if (len(approx) > 3):  
                contour_listvol[i].append(contour)

        for i2 in range (len(contour_listvol[i])): 
            (x,y),radius = cv2.minEnclosingCircle(contour_listvol[i][i2]) #finds a circle of the minimum area enclosing a 2d point set
            center = (int(x),int(y))
            radius = int(radius)
            area = cv2.contourArea(contour_listvol[i][i2]) 


            #radius<50 to indicate small objects; area check relative to radius to check the circularity
            if (radius < 50) & (area> (0.3*(3.14*(radius**2)))): #appends to boneROI if the radius is less than the specified number (indicating blood vessels)
                #append to list if it satisfies the above conditions
                boneROIvol[i].append(contour_listvol[i][i2])

        bonemwvol[i] = cv2.drawContours(r[i], boneROIvol[i],  -1, (1,0,0), 1)
    
    bonemwvol=floodfillall(bonemwvol)
    return bonemwvol

# vessels_mask=get_vessels()
# stackimages(vessels_mask)
    

In [28]:
#generate this whites mask to help guide the snake away from the thigh border and vessels (which are dark) because the snake is attracted to dark lines

def get_whites():
    white_o=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        #generate the white outside
        white_o[i]=no_skin[i].copy()
        h, w = white_o[i].shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        cv2.floodFill(white_o[i], mask, (0,0), 2);
        th, white_o[i] = cv2.threshold(white_o[i], 1, 1, cv2.THRESH_BINARY)

    #white outside and vessels
    whites=white_o+vessels_mask
    th, whites = cv2.threshold(whites, 0, 1, cv2.THRESH_BINARY)
    return whites

# whites=get_whites()
# stackimages(whites)

### Overall

## Refine Muscle Mask: Snakes


In [29]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [30]:
#initiate the snake with an initial contour
#in this case we will make a convex hull of the muscle mask (gets rid of all concavities) because this likely lies closest to the fascia

#here we find the contour of the muscle mask. the convex hull function requires a list of coordinates, but the contour is drawn in an array here for visualization

def get_contours(im):
    contoursvol=[]
    contour_listvol=[]
    maskoutlinevol=[]

    blank=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        contoursvol.append([])
        #hiercvol.append([])
        contour_listvol.append([])
        maskoutlinevol.append([])

    for i in range (j):
        a, b =  cv2.findContours(im[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contoursvol[i].append(a)

        for contour in contoursvol[i][0]:      
            contour_listvol[i].append(contour)

        #draw the colour list on a blank array
        maskoutlinevol[i] = cv2.drawContours(blank[i], contour_listvol[i],  -1, (1,0,0), 1)
    return maskoutlinevol, contoursvol

# maskoutlinevol, contoursvol=get_contours(t2)
# stackimages(maskoutlinevol)

In [31]:

def convex_hull(contours):
    hullvol=[]
    hull_demonstration = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2], 3), np.uint8)
    snakeinitiator_draw = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)
    listt=[]
    contours_prehull=[]
    for i in range(j):
        hullvol.append([])  
        listt.append([])
        contours_prehull.append([])

        for i2 in range(len(contours[i])):
            for i3 in range(len(contours[i][i2])):
                for i4 in range(len(contours[i][i2][i3])):
                    listt[i].append(contours[i][i2][i3][i4])

        contours_prehull[i]=np.array(listt[i])

        hullvol[i].append(cv2.convexHull(contours_prehull[i], False))# all 1 list of contours

        for i2 in range(len(contours[i])):
            color_contours = (0, 255, 0) # green - color for contours
            color_hull = (255, 0, 0) # red - color for convex hull
            cv2.drawContours(hull_demonstration[i], contours[i][0], i2, color_contours, 1, 8)
            cv2.drawContours(hull_demonstration[i], hullvol[i], i2, color_hull, 1, 8)


        for i2 in range(len(contours)): #??should this be contours it was just contours
            color_hull = (255, 0, 0) # blue - color for convex hull
            cv2.drawContours(snakeinitiator_draw[i], hullvol[i],  -1, (1,0,0), 1)
    return snakeinitiator_draw, hull_demonstration

# snakeinitiator_draw, hull_demonstration=convex_hull(contoursvol)
#note: convex hull intermediate step display not displaying properly for some slices, but end product is fine

def show_convexhull():
    fig, axs = plt.subplots(j, 3, figsize= (18, 120))
    for i in range(j):
        axs[i, 0].set_title(f"slice {i+1}", fontsize=18)
        axs[i, 0].imshow(maskoutlinevol[i])
        axs[i, 1].imshow(hull_demonstration[i])
        axs[i, 2].imshow(snakeinitiator_draw[i])
        
# show_convexhull()

In [32]:
# hullmask=floodfillall(snakeinitiator_draw)

In [33]:
# #connect the fragments of fascia with the hull to generate a new hull

# #begin combining the hull with the dark pieces from the otsu in order to retrieve fragments of the fascia
# def get_t3():
#     combine_hullmask_t=hullmask+t

#     th,combine_hullmask_t=cv2.threshold(combine_hullmask_t, 0, 1, cv2.THRESH_BINARY)

# #     stackimages(combine_hullmask_t)

#     t3=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
#     for i in range(j):
#         #t3[i] = (morphology.remove_small_objects(label(fascia[i]),min_size=10, connectivity=1))
#         t3[i] = (morphology.remove_small_objects(label(combine_hullmask_t[i]),min_size=10, connectivity=1))
#         #t3[i]=cv2.morphologyEx(t3[i], cv2.MORPH_CLOSE, np.ones((2,2),np.uint8))
#         t3[i]= cv2.dilate(t3[i],np.ones((4,4),np.uint8),iterations = 1)
#         t3[i] = (morphology.remove_small_objects((t3[i]).astype(bool),min_size=170, connectivity=1))
#         t3[i]= cv2.erode(t3[i],np.ones((4,4),np.uint8),iterations = 1)

#         th, t3[i]= cv2.threshold(np.uint8(t3[i]), 0, 1, cv2.THRESH_BINARY)

#     t[t==255] = 0

#     kernel = np.ones((3,3),np.uint8)

#     for i in range(j):
#         t3[i] = cv2.morphologyEx(t3[i], cv2.MORPH_OPEN, kernel)
#     return t3
# # t3=get_t3()
# # stackimages(t3)


In [34]:
#this is calf version
def get_t3():
    t_v2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        t_v2[i]=morphology.remove_small_objects(label(t[i]),min_size=300, connectivity=0)

    th, t_v2= cv2.threshold(t_v2, 0, 1, cv2.THRESH_BINARY)
    #stackimages(t_v2)

    t3=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        t3[i] = (morphology.remove_small_objects(label(t_v2[i]),min_size=10, connectivity=1))
        t3[i] = (morphology.remove_small_holes(label(t3[i]),area_threshold=2000, connectivity=1))

        t3[i]= cv2.erode(t3[i],np.ones((4,4),np.uint8),iterations = 1)
        t3[i] = (morphology.remove_small_objects(label(t3[i]),min_size=270, connectivity=1))
        t3[i]= cv2.dilate(t3[i],np.ones((4,4),np.uint8),iterations = 1)
        t3[i] = (morphology.remove_small_objects(label(t3[i]),min_size=370, connectivity=1))
        
        th, t3[i]= cv2.threshold(np.uint8(t3[i]), 0, 1, cv2.THRESH_BINARY)

    t[t==255] = 0
    kernel = np.ones((11,11),np.uint8)

    for i in range(j):

        t3[i] = cv2.morphologyEx(t3[i], cv2.MORPH_CLOSE, kernel)
        t3[i] = cv2.morphologyEx(t3[i], cv2.MORPH_OPEN, kernel)
        t3[i]= (morphology.remove_small_objects(label(t3[i]),min_size=2000, connectivity=1))
    
    
    return t3

    
    
#t3=get_t3()


In [35]:
# #begin generating new hull through the same process as above

# contoursvol=[]
# hiercvol=[]
# contour_listvol=[]
# maskoutlinevol=[]

# r=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

# for i in range(j):
#     contoursvol.append([])
#     hiercvol.append([])
#     contour_listvol.append([])
#     maskoutlinevol.append([])
    
# for i in range (j):
#     a, b =  cv2.findContours(t3[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     contoursvol[i].append(a)
#     hiercvol[i].append(b)
    
#     for contour in contoursvol[i][0]:      
#         contour_listvol[i].append(contour)

#     maskoutlinevol[i] = cv2.drawContours(r[i], contour_listvol[i],  -1, (1,0,0), 1)

# maskoutlinevol, contoursvol=get_contours(t3) #need everything above??
# stackimages(maskoutlinevol)

In [36]:
# hullvol=[]
# hull_demonstration = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2], 3), np.uint8)
# snakeinitiator_draw = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)

# listt=[]
# contours_prehull=[]
# for i in range(j):
#     hullvol.append([])  
#     listt.append([])
#     contours_prehull.append([])
    
#     for i2 in range(len(contoursvol[i])):
#         for i3 in range(len(contoursvol[i][i2])):
#             for i4 in range(len(contoursvol[i][i2][i3])):
#                 listt[i].append(contoursvol[i][i2][i3][i4])
                
#     contours_prehull[i]=np.array(listt[i])

#     hullvol[i].append(cv2.convexHull(contours_prehull[i], False))# all 1 list of contours

#     for i2 in range(len(contoursvol[i])):
#         color_contours = (0, 255, 0) # green - color for contours
#         color_hull = (255, 0, 0) # red - color for convex hull
#         cv2.drawContours(hull_demonstration[i], contoursvol[i][0], i2, color_contours, 1, 8)
#         cv2.drawContours(hull_demonstration[i], hullvol[i], i2, color_hull, 1, 8)

    
#     for i2 in range(len(contoursvol)): #check if supposed to be contoursvol instead of contours
#         color_hull = (255, 0, 0) # blue - color for convex hull
#         cv2.drawContours(snakeinitiator_draw[i], hullvol[i],  -1, (1,0,0), 1)


# snakeinitiator_draw, hull_demonstration=convex_hull(contoursvol) #need everything above??
# show_convexhull()

In [37]:
#generate full contour list #need this??
# fullcontoursvol=[]

# drawing3vol = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)

# for i in range(j):
#     fullcontoursvol.append([])
#     fullcontoursvol[i]= cv2.findContours(snakeinitiator_draw[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
# for i in range(j):
#     for i2 in range(len(fullcontoursvol[i])):
#         color = (255, 0, 0) # blue - color for convex hull
#         cv2.drawContours(drawing3vol[i], fullcontoursvol[i][0],  -1, (1,0,0), 1)   
# drawing3vol=get
# stackimages(drawing3vol)

In [38]:
#floodfill prior to combining the hulls
#check the overlap of the hulls to account for error in the hull generation

# hullmask_premerge= floodfillall(snakeinitiator_draw)
# stackimages(hullmask_premerge)

In [39]:
#adding the hulls to find the overlap

def merge_hulls():
    merged_hulls=0
    merged_conservative= np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)

    
    for i in range(j):

        merged_hulls+=hullmask_premerge[i]
    
    for i in range(j):
        merged_conservative[i]=np.uint8(merged_hulls>8)


    return merged_conservative

# merged_hulls=merge_hulls()
# stackimages(merged_hulls)

In [40]:
#generate contour for this new hull
# fullcontoursvol=[]

# merged = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)

# for i in range(j):
#     fullcontoursvol.append([])
#     fullcontoursvol[i]= cv2.findContours(merged_conservative[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
# for i in range(j):
#     for i2 in range(len(fullcontoursvol[i])):
#         color = (255, 0, 0) # blue - color for convex hull
#         cv2.drawContours(merged[i], fullcontoursvol[i][0],  -1, (1,0,0), 1)   

# merged, fullcontoursvol=get_contours(merged_hulls) #need above??
# stackimages(merged)

In [41]:
def compatible_coordlist(contours):
    hull2vol=[]

    for i in range(j):
        hull2vol.append([])
        for i2 in range(len(contours[i][0][0])):
            hull2vol[i].append(contours[i][0][0][i2][0])


        hull2vol[i]=np.array(hull2vol[i])
        hull2vol[i]=hull2vol[i].astype(float)
    return hull2vol

# hull2vol=compatible_coordlist(fullcontoursvol)
    

In [42]:
#apply the whites mask onto the filtered image to generate the image that the snake will be operating on
#apply bilateral filter to get rid of some noise
def get_snakeim():
    snake_im=Ivol8c_c_s.copy()
    for i in range(j):
        snake_im[i][whites[i]==1] = 255
    
    return snake_im

# snake_im=get_snakeim()
    #snake_im[i] = cv2.bilateralFilter(snake_im[i],20,35,35) #??why commented

# stackimages(snake_im)

In [43]:
#plans for snake: 
#have multiple snakes that have different parameters then merge them together
#because the different parameters have different abilities to converge to different shapes of fascia

In [44]:
#primary
def primary_snake():
    
    bilateral_t=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        bilateral_t[i] = cv2.bilateralFilter(snake_im[i],20,35,35)
    
    snakevol_p=[]
    for i in range(j):
        snakevol_p.append([])
        snakevol_p[i]= active_contour(bilateral_t[i], hull2vol[i],alpha=0.1, beta=1, w_line=-1, w_edge=1,gamma=0.5, convergence=0.8)
    
    #fig, axs = plt.subplots(j, 3, figsize=(18,120))   #??
#     axs[i, 0].set_title(f"slice {i+1}", fontsize=14)
#     axs[i,0].imshow(snake_im[i], cmap='bone')

#     axs[i,1].imshow(snake_im[i], cmap='bone')
#     axs[i,1].plot(hull2vol[i][:, 0], hull2vol[i][:, 1], '--r', lw=1)

#     axs[i,2].imshow(Ivol8c_roi[i], cmap='bone')
#     axs[i,2].plot(snakevol_p[i][:, 0], snakevol_p[i][:, 1], '-b', lw=1);

    return snakevol_p

# snakevol_p=primary_snake()

#fig, axs = plt.subplots(j, 3, figsize=(18,120))   #??
#     axs[i, 0].set_title(f"slice {i+1}", fontsize=14)
#     axs[i,0].imshow(snake_im[i], cmap='bone')

#     axs[i,1].imshow(snake_im[i], cmap='bone')
#     axs[i,1].plot(hull2vol[i][:, 0], hull2vol[i][:, 1], '--r', lw=1)

#     axs[i,2].imshow(Ivol8c_roi[i], cmap='bone')
#     axs[i,2].plot(snakevol_p[i][:, 0], snakevol_p[i][:, 1], '-b', lw=1);


In [45]:
def snaketomask(coordinates):
    
    vol=[]

    for i in range(j):
        vol.append([])
        for i2 in range(len(coordinates[i])):
            vol[i].append([])
            vol[i][i2].append(coordinates[i][i2])

        vol[i]=np.rint(vol[i]).astype(int)

    drawsnakevol= np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)

    #polylines function connects the dots to make it one smooth contour
    for i in range(j):
        cv2.polylines(drawsnakevol[i], [vol[i]], isClosed=True, color = (1, 0, 0) , thickness=1) 

    snakemaskvol=drawsnakevol.copy()

    h, w = subcfatvol[i].shape[:2]

    for i in range (j):
        mask = np.zeros((h+2, w+2), np.uint8)
        (x,y),radius = cv2.minEnclosingCircle(vol[i]) #(x, y) coordinates of the centre of the "circle"
        cv2.floodFill(snakemaskvol[i], mask, (round(x),round(y)), 255)


    ret, snakemaskvol = cv2.threshold(snakemaskvol,0,1,cv2.THRESH_BINARY)
    return snakemaskvol

In [46]:
#snake#2

#savior
def secondary_snake():
    snakevol_s=[]

    for i in range(j):
        snakevol_s.append([])
        snakevol_s[i]= active_contour(snake_im[i], hull2vol[i],alpha=0.0005, beta=2, w_edge=1)

    return snakevol_s
# snakevol_s=secondary_snake()


#fig, axs = plt.subplots(j, 3, figsize=(18,120)) #??
#     axs[i, 0].set_title(f"slice {i+1}", fontsize=14)
#     axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')

#     axs[i,1].imshow(Ivol8c_roi[i], cmap='bone')
#     axs[i,1].plot(hull2vol[i][:, 0], hull2vol[i][:, 1], '--r', lw=1)

#     axs[i,2].imshow(Ivol8c_roi[i], cmap='bone')
#     axs[i,2].plot(snakevol_s[i][:, 0], snakevol_s[i][:, 1], '-b', lw=1);


In [47]:
# snakemaskvol_s=snaketomask(snakevol_s)
#stackimages(snakemaskvol_s)

In [48]:
# add=snakemaskvol_p+snakemaskvol_s
# ret, add = cv2.threshold(add,0,1,cv2.THRESH_BINARY)
# stackimages(add)

In [49]:
#generate contour out of the combination of the 2 snakes, #??need all commented code?? bef and after #--------
#then use this contour as an initiator for the final snake for fine tuning
# fullcontoursvol=[]

# drawing3vol = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)

# for i in range(j):
#     fullcontoursvol.append([])
#     fullcontoursvol[i]= cv2.findContours(add[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
# for i in range(j):
#     for i2 in range(len(fullcontoursvol[i])):
#         color = (255, 0, 0) # blue - color for convex hull
#         cv2.drawContours(drawing3vol[i], fullcontoursvol[i][0],  -1, (1,0,0), 1)   


#-------------
# drawing3vol, fullcontoursvol=get_contours(add)
# hull4vol=compatible_coordlist(fullcontoursvol)
# stackimages(drawing3vol)
#---------------

# hull4vol=[]

# for i in range(j):
#     hull4vol.append([])
#     for i2 in range(len(fullcontoursvol[i][0][0])):
#         hull4vol[i].append(fullcontoursvol[i][0][0][i2][0])
        

#     hull4vol[i]=np.array(hull4vol[i])
#     hull4vol[i]=hull4vol[i].astype(float)


In [50]:
#fine tune

def snake_finetune():
    snakevol3=[]
    for i in range(j):
        snakevol3.append([])
        snakevol3[i]= active_contour(snake_im[i], hull4vol[i],alpha=0.1, beta=1, w_line=-1, w_edge=1,gamma=0.5, convergence=0.8)
    return snakevol3
        
# snakevol3=snake_finetune()
    
# fig, axs = plt.subplots(j, 3, figsize=(18,120))   
# for i in range(j):
#     axs[i, 0].set_title(f"slice {i+1}", fontsize=14)
#     axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')

#     axs[i,1].imshow(Ivol8c_roi[i], cmap='bone')
#     axs[i,1].plot(hull4vol[i][:, 0], hull4vol[i][:, 1], '-r', lw=1)

#     axs[i,2].imshow(Ivol8c_roi[i], cmap='bone')
#     axs[i,2].plot(snakevol3[i][:, 0], snakevol3[i][:, 1], '-b', lw=1);


In [51]:
# snakemaskvol=snaketomask(snakevol3)
#stackimages(snakemaskvol)

In [52]:
#adding snake and muscle roi to account for any missed pieces

# musclemaskvol2=snakemaskvol+t2
# ret, musclemaskvol2 = cv2.threshold(musclemaskvol2,0,1,cv2.THRESH_BINARY)

# fig, axs = plt.subplots (j, 4, figsize=(15,90)) #??
# for i in range(j):

#     def check2(im, roi):    
#         overlay = np.ma.masked_where(roi[i] == 0, roi[i])
#         axs[i,1].imshow(im[i], cmap="bone")
#         axs[i,1].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)
#     def check3(im, roi):    
#         overlay = np.ma.masked_where(roi[i] == 0, roi[i])
#         axs[i,2].imshow(im[i], cmap="bone")
#         axs[i,2].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)
#     def check4(im, roi):    
#         overlay = np.ma.masked_where(roi[i] == 0, roi[i])
#         axs[i,3].imshow(im[i], cmap="bone")
#         axs[i,3].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)

        
#     axs[i, 0].set_title(f"slice {i+1}", fontsize=12)   
#     axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')
#     check2(Ivol8c_roi, t2)
#     check3(Ivol8c_roi, snakemaskvol)
#     check4(Ivol8c_roi, musclemaskvol2)

In [53]:
def snake_postprocessing(): #??
    closed=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    snakemask_premerge=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        closed[i]=cv2.morphologyEx(snakemaskvol[i], cv2.MORPH_CLOSE, np.ones((13,13),np.uint8))
        snakemask_premerge[i]=cv2.morphologyEx(closed[i], cv2.MORPH_OPEN, np.ones((15,15),np.uint8))
        snakemask_premerge[i]= (morphology.remove_small_objects(label(snakemask_premerge[i]),min_size=800, connectivity=1))

    ret, snakemask_premerge = cv2.threshold(snakemask_premerge,0,1,cv2.THRESH_BINARY) 
    return snakemask_premerge

# snakemask_premerge=snake_postprocessing()
#stackimages(snakemask_premerge+musclemaskvol2)

In [54]:
#checking the overlap on the snakes
#we will keep the overlapping portions in a slice's 2 adjacent slices (3 slices total) in order to account for pieces missed
def get_snakeoverlap():
    
    snake_merge=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):

        if i==0:
            snake_merge[i]=snakemask_premerge[i]+snakemask_premerge[i+1]
            snake_merge[i][snake_merge[i]==2]=3
        elif i==(j-1):
            snake_merge[i]=snakemask_premerge[i-1]+snakemask_premerge[i]
            snake_merge[i][snake_merge[i]==2]=3
        else:
            snake_merge[i]=snakemask_premerge[i-1]+snakemask_premerge[i]+snakemask_premerge[i+1]

#     stackimages(snake_merge)

    #keeping the overlap
    ret, snake_merge= cv2.threshold(snake_merge,1,1,cv2.THRESH_BINARY)
    return snake_merge

# snake_merge=get_snakeoverlap()

In [55]:
def getmusclemask(im):
    
    premusclemaskvol=musclemaskvol=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    im_floodfillvol=im.copy()
    #im_floodfillvol=t2.copy()
         
    h, w = im.shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)

    kernel = np.ones((1,1),np.uint8)
    musclemaskvol = cv2.morphologyEx(im_floodfillvol, cv2.MORPH_CLOSE, kernel)

    cv2.floodFill(im_floodfillvol, mask, (0,0), 1);
    im_floodfillvol= cv2.dilate(im_floodfillvol,np.ones((2,2),np.uint8),iterations = 1)
    im_floodfillvol = (morphology.remove_small_holes(im_floodfillvol,area_threshold=200, connectivity=1))
    im_floodfillvol= (morphology.remove_small_objects(label(im_floodfillvol),min_size=8000, connectivity=1))
    
    
    im_floodfillvol=cv2.normalize(src=im_floodfillvol, dst=None, alpha=0.0, beta=255.0, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    th, im_floodfillvol = cv2.threshold(im_floodfillvol, 0, 1, cv2.THRESH_BINARY_INV)
    kernel = np.ones((15,15),np.uint8)
    im_floodfillvol = cv2.morphologyEx(im_floodfillvol, cv2.MORPH_OPEN, kernel)
    im_floodfillvol = cv2.morphologyEx(im_floodfillvol, cv2.MORPH_CLOSE, kernel)
    
    return im_floodfillvol


def getmusclemaskpt2():
    musclemask=np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    for i in range(j):

        musclemask[i]=getmusclemask(subcfatvol_augmented[i])
        musclemask[i]= (morphology.remove_small_objects(label(musclemask[i]),min_size=8000, connectivity=1))
    return musclemask



In [56]:
def get_defects():
    defects=snake_merge-musclemask
    kernel = np.ones((3,3),np.uint8)
    for i in range(j):
        defects[i] = cv2.morphologyEx(defects[i], cv2.MORPH_OPEN, kernel)

    defects[defects==255]=0
    
    th, defectsandbone = cv2.threshold(defects+bonemvolz, 0, 1, cv2.THRESH_BINARY)
    
    
    defectsandbone_fill=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    defectsidentify=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    defectsidentify2=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    defect_final=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        tibiacoord=np.argwhere((np.uint8(im[i]))==2) #is 2 because of its value from the label in the watershed
        tibiacoord=tibiacoord[len(tibiacoord)//2]
        defectsandbone_fill[i] = defectsandbone[i].copy()
        h, w = defectsandbone_fill[i].shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        cv2.floodFill(defectsandbone_fill[i], mask, (tibiacoord[1],tibiacoord[0]), 2)

    defectsidentify=defectsandbone_fill-(bonemvolz*2)
    defectsidentify[defectsidentify==255]=0


    for i in range(j):
        if np.amax(defectsidentify[i])==2:
            defectsidentify2+=((defectsidentify[i])//2)
        else:
            defectsidentify2+=defects[i]

    defectcoord=np.argwhere(defectsidentify2==np.amax(defectsidentify2)) 
    defectcoord=defectcoord[len(defectcoord)//2]
    for i in range(j):
        defect_final[i] = defects[i].copy()
        h, w = defectsandbone_fill[i].shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        if defects[i][defectcoord[0],defectcoord[1]]==1:
            cv2.floodFill(defect_final[i], mask, (defectcoord[1],defectcoord[0]), 2)
        else:
            pass

    defect_final=np.uint8(defect_final==2)
    defect_final_merge=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        if i==0:
            defect_final_merge[i]=defect_final[i]+defect_final[i+1]
            defect_final_merge[i][defect_final_merge[i]==2]=3
        elif i==(j-1):
            defect_final_merge[i]=defect_final[i-1]+defect_final[i]
            defect_final_merge[i][defect_final_merge[i]==2]=3
        else:
            defect_final_merge[i]=defect_final[i-1]+defect_final[i]+defect_final[i+1]

    defect_final_merge=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        defect_final_merge+=defect_final[i]
    
    

    
    
    
    return (defect_final_merge>8)



#stackimages(snake_defectremoved)


In [57]:
#island removal from above to account for overshooting...  #??

#s_dif_refined = np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
#for i in range(j):
#    s_dif_refined[i]=label(s_dif[i])
#    s_dif_refined[i] = morphology.remove_small_objects(s_dif_refined[i],min_size=2, connectivity=1)
#    ret, s_dif_refined[i] = cv2.threshold(s_dif_refined[i],0,1,cv2.THRESH_BINARY)

#stackimages(s_dif_refined)

In [58]:
#g=musclemaskvol2-t2 #??
#stackimages(g)

In [59]:
#erodesnake=musclemaskvol2-t2 #??
 
#print(t2.dtype)
#print(musclemaskvol2.dtype)
#stackimages(erodesnake)

#stackimages(musclemaskvol2)

## Bone Removal

Gaussian Filter

In [60]:
#bilateral_filter =np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') #??

#for i in range(j):
#    bilateral_filter[i] = cv2.bilateralFilter(Ivol8c_roi[i],20,35,35)
#stackimages(bilateral_filter)

In [61]:
def corticaloutline():
    cortical=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    cortical=get_thighpremask(Ivol8c_c_s)
    for i in range(j):

        cortical[i] = (morphology.remove_small_holes(cortical[i],area_threshold=40, connectivity=1))

    cortical=cortical*thighmask
    return cortical


In [62]:
#try the snake with just enhanced sharpness; need to convert from im to array with PIL
#next, could try to set a floor value so that there is decreased competition from the contour of the muscle

from PIL import ImageEnhance #add this to first cell
from PIL import Image

def get_bonemask():
    Ivol8c_boneprep=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        f=curvature_flow[i].copy()
        f=Image.fromarray(f, mode=None)
        enhancer = ImageEnhance.Sharpness(f)

        factor =  12.0

        Ivol8c_boneprep[i]=enhancer.enhance(factor)
        Ivol8c_boneprep[i] = cv2.bilateralFilter(Ivol8c_boneprep[i],20,35,35)

    Ivol8c_boneprep_mask=subcfat(Ivol8c_boneprep)

    for i in range(j):
        Ivol8c_boneprep_mask[i]= cv2.erode(Ivol8c_boneprep_mask[i],np.ones((2,2),np.uint8),iterations = 1) #prep for watershed (sure bone) and rule out connectivity with muscle fat
    return Ivol8c_boneprep_mask

# Ivol8c_boneprep_mask = get_bonemask()
# stackimages(Ivol8c_boneprep_mask)
#be wary of the object removal in the object removal in the subfat removal function. should make a separate func for bone identification. lazy mode rn.

In [63]:
#initiates bone search by finding contours similar to bone

def find_potentialbone(image):
    contoursvol=[]
    hiercvol=[]
    contour_listvol=[]
    preboneROIvol=[]
    bonemwvol=[]
    slice_center=[]
    dif=[]
    pairs=[]

    def compare_contour(contour_list, i2_1, i2_2): #can we take this function out of function??
        area1=cv2.contourArea(contour_list[i2_1])
        area2=cv2.contourArea(contour_list[i2_2])

        if area1 < area2:
            remove=i2_2
        else:
            remove=i2_1

        return remove

    r=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        contoursvol.append([])
        hiercvol.append([])
        contour_listvol.append([])
        preboneROIvol.append([])
        #boneROIvol.append([])
        bonemwvol.append([])
        slice_center.append([])
        dif.append([])
        pairs.append([])

    for i in range (j):
        a, b =  cv2.findContours(image[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contoursvol[i].append(a)
        hiercvol[i].append(b)

        for contour in contoursvol[i][0]:
            approx = cv2.arcLength(contour,True)
            area = cv2.contourArea(contour) 
            (x,y),radius = cv2.minEnclosingCircle(contour)
            radius = int(radius)

            if ((approx) > 25) & (area > 25):  #Changed Here
                contour_listvol[i].append(contour)


        for i2 in range (len(contour_listvol[i])): 

            (x,y),radius = cv2.minEnclosingCircle(contour_listvol[i][i2]) #finds a circle of the minimum area enclosing a 2d point set
            center = (int(x),int(y))
            radius = float(radius)
            approx = cv2.arcLength(contour_listvol[i][i2],True)
            area=cv2.contourArea(contour_listvol[i][i2])
            #print(approx,area, i)


            #cv2.circle(MidI82,center,radius,(0,255,0),2) #draws a circle            
            if (radius < 80) & (area>(0.8*(radius**2)) and ((area)>(0.3*(radius*approx)//2))):
            #if (radius < 60) & (area>(0.8*(radius**2))):

                slice_center[i].append(center)
                preboneROIvol[i].append(contour_listvol[i][i2])

        for i2 in range(len(slice_center[i])): #comparing contours in a slice
            slice_center[i][i2]=np.array(slice_center[i][i2])

            dif[i].append((slice_center[i][i2])-(slice_center[i][:])) #compares one with all others

            dif[i][i2]=np.delete(dif[i][i2], i2, axis=0)
            for i3 in range(len(dif[i][i2])):
                if (abs(dif[i][i2][i3][0])<20) & (abs(dif[i][i2][i3][1])<20):
                    #print(i, i2, (dif[i][i2][i3][0]), (dif[i][i2][i3][1]))
                    pairs[i].append(i2)



        bonemwvol[i] = cv2.drawContours(r[i], preboneROIvol[i],  -1, (1,0,0), 1)
    return bonemwvol
# bonemwvol=find_potentialbone()
# bonemvol=floodfillall(bonemwvol)     
# stackimages(bonemvol)   



In [64]:
#label bone objects and keep the bone based off of shape #??
def finalize_bonemask(image):
    
    channels_collated=[]
    channels_isolated=[]
    above_th=[]
    fat_th=[]
    for i in range(j):
        channels_collated.append([])
        above_th.append([])
        channels_isolated.append([])
        fat_th.append([])

        channels_collated[i]=label(image[i])
        fat_th[i]=multi_otsu(Ivol8c_roi[i])

        for ii in range(np.amax(channels_collated[i])):
            channels_isolated[i].append((channels_collated[i]==ii+1))



        for ii in range(np.amax(channels_collated[i])):
            seg=channels_isolated[i][ii]*Ivol8c_roi[i]

            seg_masked=np.ma.masked_where(seg==0,seg) #fat peeled
            seg_mean=np.mean(seg_masked) 

            if seg_mean>fat_th[i]:
                above_th[i].append(channels_isolated[i][ii])
                #print(i, seg_mean, fat_th[i])

        size1=0
        size2=0
        for ii in range(len(above_th[i])):
            if np.sum(above_th[i][ii])>size1:
                size1=np.sum(above_th[i][ii])
                c=ii
            elif np.sum(above_th[i][ii])>size2:
                size2=np.sum(above_th[i][ii])
                d=ii

        try:
            bonemvol[i]=above_th[i][c]+above_th[i][d]
        except:
            try:
                bonemvol[i]=above_th[i][c]
            except:
                pass
        

        
    return bonemvol
# bonemvolz=finalize_bonemask()
###
# stackimages(bonemvolz)


In [65]:
#finalize bone mask cortical. should be able to use the above function but cant find the difference

def finalize_bonemask_cortical(image):

    channels_isolated=[]
    above_th=[]
    channels_collated=[]
    
    
    for i in range(j):
        kernel = np.ones((15,15),np.uint8)
        image[i] = cv2.morphologyEx(image[i], cv2.MORPH_CLOSE, kernel)
        kernel = np.ones((15,15),np.uint8)
        image[i] = cv2.morphologyEx(image[i], cv2.MORPH_OPEN, kernel)
    
    
    for i in range(j):
        channels_collated.append([])
        channels_collated[i]=label(image[i])

    for i in range(j):
        above_th.append([])
        channels_isolated.append([])
        for ii in range(np.amax(channels_collated[i])):
            channels_isolated[i].append((channels_collated[i]==ii+1))



        for ii in range(np.amax(channels_collated[i])):
            seg=channels_isolated[i][ii]*Ivol8c_roi[i]

            seg_masked=np.ma.masked_where(seg==0,seg) #fat peeled
            seg_mean=np.mean(seg_masked) 

            #if seg_mean>fat_th[i]:
            if seg_mean>0:
                above_th[i].append(channels_isolated[i][ii])
                #print(i, seg_mean, fat_th[i])

        size1=0
        size2=0
        for ii in range(len(above_th[i])):
            if np.sum(above_th[i][ii])>size1:
                size1=np.sum(above_th[i][ii])
                c=ii
            elif np.sum(above_th[i][ii])>size2:
                size2=np.sum(above_th[i][ii])
                d=ii

        try:
            bonemvol[i]=above_th[i][c]+above_th[i][d]
        except:
            try:
                bonemvol[i]=above_th[i][c]
            except:
                pass

    ###
    return bonemvol


In [66]:
def merge_bones():
    t_bone=cortical-bonemvols

    th, t = cv2.threshold(t_bone+skin_fat, 0, 1, cv2.THRESH_BINARY_INV)
    
    for i in range(j):
        h, w = t_bone[0].shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        cv2.floodFill(t[i], mask, (0,0), 0);
        t[i]=cv2.morphologyEx(t[i], cv2.MORPH_CLOSE, np.ones((3,3),np.uint8))
        t[i]=cv2.morphologyEx(t[i], cv2.MORPH_OPEN, np.ones((3,3),np.uint8))
        t[i] = (morphology.remove_small_objects(label(t[i]),min_size=80, connectivity=1))
        th, t[i] = cv2.threshold(t[i], 0, 1, cv2.THRESH_BINARY) 
        t[i]=cv2.morphologyEx(t[i], cv2.MORPH_CLOSE, np.ones((25,25),np.uint8))
        t[i]=cv2.morphologyEx(t[i], cv2.MORPH_OPEN, np.ones((5,5),np.uint8))

     

    p=(t+cortical)==2
    p2=cortical-p+skin_fat
    th, p2 = cv2.threshold(p2, 0, 1, cv2.THRESH_BINARY)
    
    
    
    
    
    corticalt=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):   
        corticalt[i] = (morphology.remove_small_holes(label(p2[i]),area_threshold=40, connectivity=0))
        
        corticalt[i]=cv2.morphologyEx(corticalt[i], cv2.MORPH_CLOSE, np.ones((7,7),np.uint8))
        corticalt[i]=cv2.morphologyEx(corticalt[i], cv2.MORPH_OPEN, np.ones((7,7),np.uint8))


    corticalt = corticalt+skin_fat
    th, corticalt = cv2.threshold(corticalt, 0, 1, cv2.THRESH_BINARY)
    
    return corticalt
   

In [67]:
#recovering the fibula
def recoverfibula():
    bonemvol=np.zeros([Ivol8c_roi.shape[0],Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    channels_collated=[]
    
    for i in range(j):
        channels_collated.append([])
        channels_collated[i]=label(bonemvols[i])

    channels_isolated=[]
    above_th=[]

    for i in range(j):
        above_th.append([])
        channels_isolated.append([])
        for ii in range(np.amax(channels_collated[i])):
            channels_isolated[i].append((channels_collated[i]==ii+1))



        for ii in range(np.amax(channels_collated[i])):
            seg=channels_isolated[i][ii]*Ivol8c_roi[i]

            seg_masked=np.ma.masked_where(seg==0,seg) #fat peeled
            seg_mean=np.mean(seg_masked) 

            #if seg_mean>fat_th[i]:
            if seg_mean>0:
                above_th[i].append(channels_isolated[i][ii])
                #print(i, seg_mean, fat_th[i])

        size1=0
        size2=0
        for ii in range(len(above_th[i])):
            if np.sum(above_th[i][ii])>size1:
                size1=np.sum(above_th[i][ii])
                c=ii
            elif np.sum(above_th[i][ii])>size2:
                size2=np.sum(above_th[i][ii])
                d=ii

        try:
            bonemvol[i]=above_th[i][d]
        except: 
            pass
    return bonemvol

###


###____
def recoverfibulacortical():
    marrowcount=0
    for i in range(j):
        marrowcount+=fibulamarrow[i]
    if np.amax(marrowcount)<(j*0.7):
        recoveredmarrow=0
    else:
        fibulaoverlay=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

        for i in range(j):
            fibulaoverlay+=fibulamarrow[i]
        fibulaoverlaymax=fibulaoverlay==np.amax(fibulaoverlay)


        centerfibmarrowlist=np.argwhere(fibulaoverlaymax==1)
        centerfibmarrow=centerfibmarrowlist[len(centerfibmarrowlist)//2]

        ###___
        #for other code, cannot use fg. need to use boneprep to get rid of as much fat as possible
        recoveredmarrow=fg.copy()
        unsuccessfulfib=[]
        successfulfib=[]

        for i in range(j):
            h, w = Ivol8c_roi[0].shape[:2]
            mask = np.zeros((h+2, w+2), np.uint8)
            if recoveredmarrow[i][centerfibmarrow[0],centerfibmarrow[1]]==1:
                cv2.floodFill(recoveredmarrow[i], mask, (centerfibmarrow[1],centerfibmarrow[0]), 2);
                ret, recoveredmarrow[i] = cv2.threshold(recoveredmarrow[i],1,1,cv2.THRESH_BINARY)
                successfulfib.append(i)
            else:
                unsuccessfulfib.append(i)
        #while loop to iterate through all until done?

        for item in unsuccessfulfib:
            overlaySUS=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
            for item2 in successfulfib:#overlay the recovered marrow onto unsuccessful bone masks
                overlaySUS+=recoveredmarrow[item2]
            overlaySUS=(overlaySUS>0)*len(successfulfib)


        for item in unsuccessfulfib:
            overlaySUS2=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')   
            overlaySUS2=overlaySUS+recoveredmarrow[item]

            coord=np.argwhere((recoveredmarrow[item])==1)

            maximum=0
            for item2 in coord:
                if overlaySUS2[item2[0],item2[1]]>maximum:
                    maximum=overlaySUS2[item2[0],item2[1]]

            repairUScoord=np.argwhere(overlaySUS2==maximum)
            repairUScoord=repairUScoord[len(repairUScoord)//2]

            h, w = Ivol8c_roi[0].shape[:2]
            mask = np.zeros((h+2, w+2), np.uint8)
            print(recoveredmarrow[item][repairUScoord[1],repairUScoord[0]])
            cv2.floodFill(recoveredmarrow[item], mask, (repairUScoord[1],repairUScoord[0]), 2);
            ret, recoveredmarrow[item] = cv2.threshold(recoveredmarrow[item],1,1,cv2.THRESH_BINARY)


    return recoveredmarrow

fibulaoverlay=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

for i in range(j):
    fibulaoverlay+=fibulamarrow[i]
fibulaoverlaymax=fibulaoverlay==np.amax(fibulaoverlay)


centerfibmarrowlist=np.argwhere(fibulaoverlaymax==1)
centerfibmarrow=centerfibmarrowlist[len(centerfibmarrowlist)//2]

###___
#for other code, cannot use fg. need to use boneprep to get rid of as much fat as possible
recoveredmarrow=fg.copy()
unsuccessfulfib=[]
successfulfib=[]

for i in range(j):
    h, w = Ivol8c_roi[0].shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    if recoveredmarrow[i][centerfibmarrow[0],centerfibmarrow[1]]==1:
        cv2.floodFill(recoveredmarrow[i], mask, (centerfibmarrow[1],centerfibmarrow[0]), 2);
        ret, recoveredmarrow[i] = cv2.threshold(recoveredmarrow[i],1,1,cv2.THRESH_BINARY)
        successfulfib.append(i)
    else:
        unsuccessfulfib.append(i)
#while loop to iterate through all until done?

for item in unsuccessfulfib:
    overlaySUS=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    for item2 in range(len(successfulfib)):#overlay the recovered marrow onto unsuccessful bone masks
        overlaySUS+=recoveredmarrow[item2]
        
    plt.imshow(overlaySUS)
        
    overlaySUS+=recoveredmarrow[item]

    coord=np.argwhere((recoveredmarrow[item])==1)

    maximum=0
    for item2 in coord:
        if overlaySUS[item2[0],item2[1]]>maximum:
            maximum=overlaySUS[item2[0],item2[1]]

    repairUScoord=np.argwhere(overlaySUS==maximum)
    repairUScoord=repairUScoord[len(repairUScoord)//2]

    h, w = Ivol8c_roi[0].shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    print(recoveredmarrow[item][repairUScoord[1],repairUScoord[0]])
    cv2.floodFill(recoveredmarrow[item], mask, (repairUScoord[1],repairUScoord[0]), 2);
    ret, recoveredmarrow[item] = cv2.threshold(recoveredmarrow[item],1,1,cv2.THRESH_BINARY)
    

stackimages(recoveredmarrow)

for item in unsuccessfulfib:
    overlaySUS=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for item2 in range(len(successfulfib)):#overlay the recovered marrow onto unsuccessful bone masks
        overlaySUS+=recoveredmarrow[item2]
    overlaySUS=(overlaySUS>0)*len(successfulfib)
    

for item in unsuccessfulfib:
    overlaySUS2=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')   
    overlaySUS2=overlaySUS+recoveredmarrow[item]
    
    coord=np.argwhere((recoveredmarrow[item])==1)

    maximum=0
    for item2 in coord:
        if overlaySUS2[item2[0],item2[1]]>maximum:
            maximum=overlaySUS2[item2[0],item2[1]]

    repairUScoord=np.argwhere(overlaySUS2==maximum)
    repairUScoord=repairUScoord[len(repairUScoord)//2]

    h, w = Ivol8c_roi[0].shape[:2]
    mask = np.zeros((h+2, w+2), np.uint8)
    print(recoveredmarrow[item][repairUScoord[1],repairUScoord[0]])
    cv2.floodFill(recoveredmarrow[item], mask, (repairUScoord[1],repairUScoord[0]), 2);
    ret, recoveredmarrow[item] = cv2.threshold(recoveredmarrow[item],1,1,cv2.THRESH_BINARY)
stackimages(recoveredmarrow)


#recovering the fibula
def recoverfibula():
    bonemvol=np.zeros([Ivol8c_roi.shape[0],Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    channels_collated=[]
    
    for i in range(j):
        channels_collated.append([])
        channels_collated[i]=label(bonemvols[i])

    channels_isolated=[]
    above_th=[]

    for i in range(j):
        above_th.append([])
        channels_isolated.append([])
        for ii in range(np.amax(channels_collated[i])):
            channels_isolated[i].append((channels_collated[i]==ii+1))



        for ii in range(np.amax(channels_collated[i])):
            seg=channels_isolated[i][ii]*Ivol8c_roi[i]

            seg_masked=np.ma.masked_where(seg==0,seg) #fat peeled
            seg_mean=np.mean(seg_masked) 

            #if seg_mean>fat_th[i]:
            if seg_mean>0:
                above_th[i].append(channels_isolated[i][ii])
                #print(i, seg_mean, fat_th[i])

        size1=0
        size2=0
        for ii in range(len(above_th[i])):
            if np.sum(above_th[i][ii])>size1:
                size1=np.sum(above_th[i][ii])
                c=ii
            elif np.sum(above_th[i][ii])>size2:
                size2=np.sum(above_th[i][ii])
                d=ii

        try:
            bonemvol[i]=above_th[i][d]
        except: 
            pass
    return bonemvol

###


###____
def recoverfibulacortical():
    fibulaoverlay=np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        fibulaoverlay+=fibulamarrow[i]
    fibulaoverlaymax=fibulaoverlay==np.amax(fibulaoverlay)


    centerfibmarrowlist=np.argwhere(fibulaoverlaymax==1)
    centerfibmarrow=centerfibmarrowlist[len(centerfibmarrowlist)//2]

    ###___
    #for other code, cannot use fg. need to use boneprep to get rid of as much fat as possible
    recoveredmarrow=fg.copy()
    unsuccessfulfib=[]
    successfulfib=[]

    for i in range(j):
        h, w = Ivol8c_roi[0].shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        if recoveredmarrow[i][centerfibmarrow[0],centerfibmarrow[1]]==1:
            cv2.floodFill(recoveredmarrow[i], mask, (centerfibmarrow[1],centerfibmarrow[0]), 2);
            ret, recoveredmarrow[i] = cv2.threshold(recoveredmarrow[i],1,1,cv2.THRESH_BINARY)
            successfulfib.append(i)
        else:
            unsuccessfulfib.append(i)
    #while loop to iterate through all until done?

    p=0
    overlaySUS=0

    for i in successfulfib: #note: this loop intentionally only adds the successfully isolated fibulas
        p+=recoveredmarrow[i]

    p=(p>0)*len(successfulfib) #all succesfully isolated marrows are merged and assigned a value of the number of successful slices

    overlaySUS=p.copy()

    for i in unsuccessfulfib: #note: this loop intentionally only adds the UNsuccessfully isolated fibulas
        overlaySUS+=recoveredmarrow[i]


    for item in unsuccessfulfib:

        coord=np.argwhere((overlaySUS)==np.amax(overlaySUS))

        repairUScoord=coord[len(coord)//2]

        h, w = Ivol8c_roi[0].shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)
        if recoveredmarrow[item][repairUScoord[1],repairUScoord[0]]==1:
            cv2.floodFill(recoveredmarrow[item], mask, (repairUScoord[1],repairUScoord[0]), 2);
            ret, recoveredmarrow[item] = cv2.threshold(recoveredmarrow[item],1,1,cv2.THRESH_BINARY)
        else:
            print("error on", item,", no overlap")




    return recoveredmarrow

In [68]:
def cleanup_bone(image):
    bonemvolz=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        kernel = np.ones((15,15),np.uint8)
        bonemvolz[i] = cv2.morphologyEx(image[i], cv2.MORPH_CLOSE, kernel)
        kernel = np.ones((9,9),np.uint8)
        bonemvolz[i] = cv2.morphologyEx(bonemvolz[i], cv2.MORPH_OPEN, kernel)
    return bonemvolz


In [69]:
#watershedding the bone to make it conform better to bone boundaries

def watershed_bone():
    bg=markers=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fg=markers=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    
    xx=0
    for i in range(j):
        xx+=bonemvolz[i]
    xx=np.uint8(xx>5)
    
    for i in range(j):   
        bg[i]= cv2.dilate(xx,np.ones((4,4),np.uint8),iterations = 2)
        fg[i]= cv2.erode(xx,np.ones((7,7),np.uint8),iterations = 1)

    unsure=bg-fg

    for i in range(j):
        markers[i][unsure[i]==1] = 255
        markers[i]=markers[i]+1
        markers[i][markers[i]==256] = 0

        markers[i]=label(markers[i])
    
    backtorgb=[]
    im=[]

    markers=np.int32(markers)

    for i in range(j):
        backtorgb.append([])
        im.append([])
        backtorgb[i] = cv2.cvtColor(bilateral_filter[i],cv2.COLOR_GRAY2RGB)

    for i in range(j):

        im[i] = cv2.watershed(backtorgb[i],markers[i])
    return im

# im=watershed_bone()
# stackimages(im)

In [70]:
def roimask():
    t=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j):
        t[i]=im[i]==2
        t[i]+=im[i]==3
    roimaskvol=snakemaskvol-t
    roimaskvol[roimaskvol==255]=0
    ret, roimaskvol = cv2.threshold(roimaskvol,0,1,cv2.THRESH_BINARY)
    
    
    
    return roimaskvol
    
# roimaskvol=roimask() 
#stackimages(roimaskvol)



In [71]:
# fig, axs = plt.subplots(j, 2, figsize=(12, j*6)) #turned this into function!!
# for i in range(j):

#     def check2(im, roi):    
#         overlay = np.ma.masked_where(roi[i] == 0, roi[i])
#         axs[i,1].imshow(im[i], cmap="bone")
#         axs[i,1].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)
        
#     axs[i, 0].set_title(f"slice {i+1}", fontsize=12)   
#     axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')
#     check2(Ivol8c_roi, roimaskvol)

In [72]:
#works
def check_roi():
    def check2(im, roi):    
        overlay = np.ma.masked_where(roi[i] == 0, roi[i])
        axs[i,0].set_title(f"slice {i+1}", fontsize=12)   
        axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')
        axs[i,1].imshow(im[i], cmap="bone")
        axs[i,1].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)
    fig, axs = plt.subplots(j, 2, figsize=(12, j*6))   
    for i in range(j):
        check2(Ivol8c_roi, roimaskvol)
# check_roi()

In [73]:
#roivol=roimaskvol*Ivol8c_roi
# roivol=roimaskvol*Ivol8c_roi
# stackimages(roivol)

## Round 1: Fat segmentation

### Threshold optimization #1

In [74]:
#getinitial threshold for optimization loop using multi-otsu
def initial_th(image):
    initial_th=[]

    for i in range(j):
        initial_th.append([])
        initial_th[i]=multi_otsu(image[i]) #need [i] here ,won't work if call roivol[i]
#         print (initial_th[i])
    return initial_th

# initial_th1_R1=initial_th(roivol)



In [75]:
def optimize_threshold(roi, ots): #not (fatth,roi) cus fatth needs to change with every slice
  
    k=1
    ThPrev=0
    ThRev=ots  #starting point! we compare newest threshold to prev one

    #New lists
    ThPrevlist=[ThPrev]  #include starting ThPrev - for table later
    ThRevlist=[ThRev] #include starting ThRev  - for table and graph later

    klist=[0] #iteratios - include 0th iteration - for table and graph later
    matchlist=[] #ThRev=ThPrev yes or no - for table later
    while ThRev!=ThPrev:  #while new threshold is NOT equal to prev threshold
        
        ThPrev=ThRev#make new threshold=prev threshold, new new threshold will be created at the end of loop
        
        prefatmask = label(roi>ThRev) 
        prefatmask = np.uint8(prefatmask) 
        prefatmask = (morphology.remove_small_objects(prefatmask,min_size=8, connectivity=1))
        ret, fatmask = cv2.threshold(prefatmask,0,1,cv2.THRESH_BINARY)

        fatseg = fatmask*roi #roi=roivol[i] later
        preMuscSegM=roi-fatseg #muscle-fat=muscle
        MuscSegM=np.ma.masked_where(preMuscSegM == 0, preMuscSegM)#muscle peeled
        FatSegM=np.ma.masked_where(fatseg==0,fatseg) #fat peeled
        MuscSegI=np.mean(MuscSegM) #mean signal intensity
        FatSegI=np.mean(FatSegM)
        ThRev=(1+((FatSegI-MuscSegI)/FatSegI))*MuscSegI #NOT MUSCLE-FAT
        
        ThPrevlist.append(ThPrev) 
        ThRevlist.append(ThRev) #append ThRev, starting from first iteration
        klist.append(k) #append k value, starting k=1 (first iteration)
        matchlist.append("No") 
        k+=1
        if k==50:
            break
       
        if ThRev==ThPrev:
            matchlist.append("Yes")
            table=QTable([klist,ThPrevlist,ThRevlist,matchlist],
            names=('Iteration','ThPrev','ThRev','ThRev=ThPrev?'))
#             print (table)

            x=klist #iterations
            y=ThRevlist
#             plt.plot(x,y)

#             plt.xlabel ('Iterations')
#             plt.ylabel('Fat Threshold')
#             plt.show()
    return fatmask, fatseg, ThRev, MuscSegI #can we delete muscSegI
      

In [76]:
def opt_Th1R1(): #optimized Th1 Round 1
    fatseg1vol_mask_R1=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg1vol_R1=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    thresholds=[] 

    for i in range(j): 
        fatseg1vol_mask_R1[i],fatseg1vol_R1[i], ThRev, MuscSegI=optimize_threshold(roivol[i], initial_th1_R1[i])
        thresholds.append(ThRev) 
#         print(f"\nSlice {i+1} Optimized Threshold #1 ={thresholds[i]}\n--------------------------------------------------------------------\n") #i+1 cus slice 1=index0, slice 15=index 14
    
    for i in range(j):
        print (f"File {files} Slice {i+1} Optimized Threshold #1 ={thresholds[i]}") #note added "files"
        
    return thresholds,fatseg1vol_mask_R1,fatseg1vol_R1
        
# Th1_R1,fatseg1vol_mask_R1,fatseg1vol_R1=opt_Th1R1() #get optimized thresholds 1 for Round 1

In [77]:
# stackimages(fatseg1vol_R1)

### Threshold optimization #2

In [78]:
#Remove the first round of fat

# change name here
def subtract_fat1(fat1):
    roi2vol=np.empty([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    roi2vol=roivol-fat1 #same as for i range(j): roi2vol[i]=roivol[i]-fat1[i]
    return roi2vol

# roi2vol=subtract_fat1(fatseg1vol_R1)


In [79]:
# initial_th2_R1=initial_th(roi2vol)

In [80]:
def opt_Th2R1():
    fatseg2vol_mask=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    fatseg2vol=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

    thresholds2=[]
    for i in range(j): 
        fatseg2vol_mask[i],fatseg2vol[i], ThRev, MuscSegI=optimize_threshold(roi2vol[i], initial_th2_R1[i])
        thresholds2.append(ThRev) #fatth will be its own list item later. fatth is just a place-holder
#         print(f"\nSlice {i+1} Optimized Threshold #2 ={thresholds2[i]}\n--------------------------------------------------------------------\n") #i+1 cus slice 1=index0, slice 15=index 14
    for i in range(j):
        print(f"File {files} Slice {i+1} Optimized Threshold #2 ={thresholds2[i]}") #note added "files"
    return thresholds2
# Th2_R1=opt_Th2R1()

In [81]:
#apply Th2 to get fat from Th1

def fatfinal_R1(): #apply Th2 R1 - this is final results NOT checking for Z-connectivity - used this for Z-connectivity check OG and revised methods (not for Kenneth's)

    fatsegfinalvol_mask_R1=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    fatsegfinalvol_R1=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
    for i in range(j):
        fatsegfinalvol_mask_R1[i] = label(roivol[i]>Th2_R1[i]) #changed from thresholds2
        fatsegfinalvol_mask_R1[i] = np.uint8(fatsegfinalvol_mask_R1[i]) 
        fatsegfinalvol_mask_R1[i] = (morphology.remove_small_objects(fatsegfinalvol_mask_R1[i],min_size=8, connectivity=1)).astype(int)
        ret, fatsegfinalvol_mask_R1[i] = cv2.threshold(fatsegfinalvol_mask_R1[i],0,1,cv2.THRESH_BINARY) 

        fatsegfinalvol_R1[i]= fatsegfinalvol_mask_R1[i]*roivol[i]   
#         print (f"Slice {i+1} Area= {np.sum(fatsegfinalvol_R1[i]>0)}") #for testing?
    return fatsegfinalvol_mask_R1,fatsegfinalvol_R1

# fatsegfinalvol_mask_R1,fatsegfinalvol_R1=fatfinal_R1()
# stackimages(fatsegfinalvol_mask_R1)
# stackimages(fatsegfinalvol_R1)


In [82]:
def Th2finefat():
    fatseg2_refinedvol_mask= (fatsegfinalvol_mask_R1 != fatseg1vol_mask_R1) #finer fat from Th2 loop not captured in Th1 loop
    fatseg2_refinedvol=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8') #added!
    
    for i in range(j):
        fatseg2_refinedvol[i]=fatseg2_refinedvol_mask[i]*roivol[i]
    return fatseg2_refinedvol_mask,fatseg2_refinedvol

# fatseg2_refinedvol_mask,fatseg2_refinedvol=Th2finefat()
# stackimages(fatseg2_refinedvol)


## Round 2: Fat Segmention + Z-Connectivity Check

#### Threshold Optimization #1

In [83]:
def apply_th(th): #apply th to get images to be used for z-connectivity checking
    zcheck=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    for i in range(j): 
        zcheck[i]=(roivol[i]>th[i]).astype(int)
    return zcheck

# zcheck_Th2R1=apply_th(Th2_R1) #for kenneth's

# for i in range(j):
#     print (np.sum(zcheck_Th2R1[i]))

In [84]:
def optimize_thresholdvol_R2(i, roivar, initial_th, zcheck): #changes ots to initial_th - note initial th for R2 Th1 is same as R1 Th1 (multiotsu from roivol)
     
    k=1
    ThPrev_R2=0 
    ThRev_R2= initial_th[i] 
    x=0
    y=0
    
    #New lists
    ThPrevlist=[ThPrev_R2]  #include starting ThPrev - for table later
    ThRevlist=[ThRev_R2] #include starting ThRev  - for table and graph later

    klist=[0] #iteratios - include 0th iteration - for table and graph later
    matchlist=[] #ThRev=ThPrev yes or no - for table later
    
    while ThRev_R2!=ThPrev_R2:
        ThPrev_R2=ThRev_R2

        prefatmask_R2 = (roivar[i]>ThRev_R2)
        prefatmask_R2 = np.uint8(prefatmask_R2)
        ret, fatmask_R2 = cv2.threshold(prefatmask_R2,0,1,cv2.THRESH_BINARY)  #need this??

        if i==0:
            fatcombined_next=zcheck[i+1]+fatmask_R2
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_next
   
        elif i==(j-1):
            fatcombined_prev=zcheck[i-1]+fatmask_R2
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            z_connection=fatconnectedparts_prev
   
        else:
            fatcombined_prev=zcheck[i-1]+fatmask_R2
            fatcombined_next=fatmask_R2+zcheck[i+1]
            ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
            ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

            z_connection=fatconnectedparts_prev+fatconnectedparts_next
            ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY) 
        
        #NEW: find XY connections to Z connected parts
        coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
        im_ff=fatmask_R2.copy()
        h, w = im_ff.shape[:2] #added
        mask = np.zeros((h+2, w+2), np.uint8)
        for item in range(len(coordinates)):
            cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)
        
        #Remove small islands for Non-Z parts
        nonZ =label(im_ff==1)
        nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
        ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)

        Z = (im_ff==2)
  
        prefatseg1_R2=(Z+nonZ_keep) #do we need int cus boolean??
        fatseg1_R2=prefatseg1_R2*Ivol8c_roi[i]
        #Fat and Muscle Quantification
        preMuscSegP_R2=roivol[i]-fatseg1_R2
        MuscSegP_R2=np.ma.masked_where(preMuscSegP_R2 == 0, preMuscSegP_R2)
        FatSegP_R2=np.ma.masked_where(fatseg1_R2==0,fatseg1_R2) 
        MuscSegI_R2=np.mean(MuscSegP_R2)
        FatSegI_R2=np.mean(FatSegP_R2)
        ThRev_R2=(1+((FatSegI_R2-MuscSegI_R2)/FatSegI_R2))*MuscSegI_R2 

        ThPrevlist.append(ThPrev_R2) 
        ThRevlist.append(ThRev_R2) #append ThRev_R2, starting from first iteration
        klist.append(k) #append k value, starting k=1 (first iteration)
        matchlist.append("No")
        
        k+=1
        if k==50:
            break
            
        if ThRev_R2==ThPrev_R2:
            matchlist.append("Yes")
            table=QTable([klist,ThPrevlist,ThRevlist,matchlist],
            names=('Iteration','ThPrev_R2','ThRev_R2','ThRev_R2=ThPrev_R2?'))
#             print (table)

#             x=klist #iterations
#             y=ThRevlist
#             plt.plot(x,y)

#             plt.xlabel ('Iterations')
#             plt.ylabel('Fat Threshold')
#             plt.show()

    #  if i==8:
    #       fig, axs = plt.subplots (1,9, figsize=(15,10))
    #       axs[0].imshow(zcheck[i-1],cmap="bone")
    #       axs[0].set_title(f"Slice {i} R1")     
    #       axs[1].imshow(fatmask_R2,cmap="bone")
    #       axs[1].set_title(f"Slice {i+1} raw threshold")
    #       axs[2].imshow(zcheck[i+1],cmap="bone")
    #       axs[2].set_title(f"Slice {i+2} raw threshold")

    #       axs[3].imshow(im_ff)
    #       axs[3].set_title(f"Slice {i+1} SOI FF")
    #       axs[4].imshow(Z,cmap="bone")
    #       axs[4].set_title(f"Slice {i+1} Z")
    #       axs[5].imshow(nonZ,cmap="bone")
    #       axs[5].set_title(f"Slice {i+1} nonZ")
    #       axs[6].imshow(nonZ_keep,cmap="bone")
    #       axs[6].set_title(f"Slice {i+1} nonZ_keep")
    #       axs[7].imshow(prefatseg1_R2,cmap="bone")
    #       axs[7].set_title(f"Slice {i+1} R2 Final Mask")
    #       axs[8].imshow(fatseg1_R2,cmap="bone")
    #       axs[8].set_title(f"Slice {i+1} R2 Final ")
    # fig, axs = plt.subplots (1,7, figsize=(27,20))  
    # if i>0:  
    #     axs[0].imshow(zcheck[i-1],cmap="bone")
    # else:
    #     axs[0].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone")
    # axs[0].set_title(f"Slice {i-2} R1")     
    # axs[1].imshow(fatmask_R2,cmap="bone")
    # axs[1].set_title(f"Slice {i+1} raw threshold")
    
    # if i<(j-1):  
    #     axs[2].imshow(zcheck[i+1],cmap="bone")
    # else:
    #     axs[2].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone")           
    # axs[2].set_title(f"Slice {i+2} R1")
    # axs[3].imshow(im_ff)
    # axs[3].set_title(f"Slice {i+1} SOI FF")
    # axs[4].imshow(prefatseg1_R2,cmap="bone")
    # axs[4].set_title(f"Slice {i+1} R2 Final Mask")
    # axs[5].imshow(fatseg1_R2,cmap="bone")
    # axs[5].set_title(f"Slice {i+1} R2 Final ")
    # axs[6].imshow(fatsegfinalvol_mask_R1[i],cmap="bone")
    # axs[6].set_title(f"Slice {i+1} R1 Final ")
    thresholds_R2=ThRev_R2
    return prefatseg1_R2, fatseg1_R2, thresholds_R2

In [85]:
def opt_Th1R2(roivar, initial_th, zcheck):
    fatseg1vol_mask_R2 =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg1vol_R2 =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') #change name

    thresholds_R2 =[] 
    for i in range(j):
        thresholds_R2 .append([])
        fatseg1vol_mask_R2[i],fatseg1vol_R2[i], thresholds_R2[i]=optimize_thresholdvol_R2(i, roivar,initial_th,zcheck)  #fatsegfinalvol_mask_R1=round 1 final fat mask with islands REMOVED
#         print (f"Slice {i+1} Th1 R2 = {thresholds_R2[i]}")
#         print ("-----------------------------------------------------------")

    for i in range(j):
        print (f"File {files} Slice {i+1} Th1 R2 ={thresholds_R2 [i]}") #note added "files"
    return fatseg1vol_mask_R2 ,fatseg1vol_R2 , thresholds_R2 



In [86]:
#Original + Original Revised
# fatseg1vol_mask_R2_OG_rev,fatseg1vol_R2_OG_rev, thresholds_R2_OG_rev=opt_Th1R2(roivol,initial_th1_R1,fatsegfinalvol_mask_R1) #note initial_th1_R1 is used for R2 cus they same value

In [87]:
#Kenneth's
#Th1 R2
# fatseg1vol_mask_R2_ken,fatseg1vol_R2_ken, thresholds_R2_ken=opt_Th1R2(roivol,initial_th1_R1,zcheck_Th2R1)

#### Threshold Optimization #2

In [88]:
#Original + Original Revised
# roi2vol_R2_OG_rev=subtract_fat1(fatseg1vol_R2_OG_rev) #roivol-fatseg1vol_R2_OG_rev
#stackimages(roi2vol_R2_OG_rev)

In [89]:
#Kenneth's
# roi2vol_R2_ken=subtract_fat1(fatseg1vol_R2_ken) #roivol-fatseg1vol_R2_ken

#stackimages(roi2vol_R2_ken)

In [90]:
#Original + Original Revised-Get Initial Th2 loop R2
# initial_th2_R2_OGrev=initial_th(roi2vol_R2_OG_rev)


In [91]:
def opt_Th2R2(roivar, initial_th, zcheck): #note code =opt_Th1R1
    fatseg1vol_mask_R2 =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg1vol_R2 =np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8') #change name

    thresholds_R2 =[] 
    for i in range(j):
        thresholds_R2 .append([])
        fatseg1vol_mask_R2[i],fatseg1vol_R2[i], thresholds_R2[i]=optimize_thresholdvol_R2(i, roivar,initial_th,zcheck)  #fatsegfinalvol_mask_R1=round 1 final fat mask with islands REMOVED
#         print (f"Slice {i+1} Th1 R2 = {thresholds_R2[i]}")
#         print ("-----------------------------------------------------------")

    for i in range(j):
        print (f"File {files} Slice {i+1} Th2 R2 ={thresholds_R2 [i]}") #note added "files"
    return fatseg1vol_mask_R2 ,fatseg1vol_R2 , thresholds_R2 


In [92]:
#Original + Original Revised-Get optimized Th2 R2 

# fatseg2vol_R2_OG_rev,fatseg2vol_mask_R2_OG_rev, thresholds2_R2_OG_rev=opt_Th2R2(roi2vol_R2_OG_rev, initial_th2_R2_OGrev, fatsegfinalvol_mask_R1)

In [93]:
#Kenneth's - get initial Th2 R2

# initial_th2_R2_ken=initial_th(roi2vol_R2_ken)

# for i in range(j):
#     print (initial_th2_R2_k[i])

In [94]:
#Kenneth's -optimize Th2 R2
# fatseg2vol_R2_ken,fatseg2vol_mask_R2_ken,thresholds2_R2_ken=opt_Th2R2(roi2vol_R2_ken, initial_th2_R2_ken, zcheck_Th2R1)

## Round 3: Final Z-Connectivity Check

In [95]:
#Original Revised-Apply Th2 R2 (don't need for OG)

# zcheck_Th2R2_rev=apply_th(thresholds2_R2_OG_rev) #get roivol[i]>thresholds2_R2_OG_rev[i]  


In [96]:
#Kenneth's
# zcheck_Th2R2_ken=apply_th(thresholds2_R2_ken)


In [97]:
def R3_zcheck(i,th,zcheck): #need i in parameter or else won't differentiate when call this fxn in another fxn 
    prefatmask_R3 = (roivol[i]>th[i]) 
#     print (th[i]) #this is ok when change th[i] to th
#     plt.imshow(roivol[i],"bone")
#     plt.show()
#     plt.imshow(prefatmask_R3,"bone")
#     plt.show()
    fatmask_R3 = np.uint8(prefatmask_R3)
    
    if i==0:
        fatcombined_next=zcheck[i+1]+fatmask_R3
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_next      
    elif i==(j-1):
        fatcombined_prev=zcheck[i-1]+fatmask_R3  
        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        z_connection=fatconnectedparts_prev    
    else:
        fatcombined_prev=zcheck[i-1]+fatmask_R3
        fatcombined_next=zcheck[i+1]+fatmask_R3 

        ret, fatconnectedparts_prev= cv2.threshold(fatcombined_prev,1,1,cv2.THRESH_BINARY)
        ret, fatconnectedparts_next= cv2.threshold(fatcombined_next,1,1,cv2.THRESH_BINARY)

        z_connection=fatconnectedparts_prev+fatconnectedparts_next
        ret, z_connection= cv2.threshold(z_connection,0,1,cv2.THRESH_BINARY)      

        
    coordinates= np.argwhere(z_connection ==1) #put coordinates of Z-connections into a list      
    im_ff=fatmask_R3.copy()
    h, w = im_ff.shape[:2] 
    mask = np.zeros((h+2, w+2), np.uint8)
    for item in range(len(coordinates)):
        cv2.floodFill(im_ff, mask, (coordinates[item][1],coordinates[item][0]), 2)

    #Remove small islands for Non-Z parts
    nonZ =label(im_ff==1)
    nonZ_keep = (morphology.remove_small_objects(nonZ,min_size=8, connectivity=1))
    ret, nonZ_keep= cv2.threshold(np.uint8(nonZ_keep),0,1,cv2.THRESH_BINARY)

    Z = (im_ff==2)
   
    final_fat_mask=(Z+nonZ_keep) 
    final_fat=final_fat_mask*Ivol8c_roi[i] 
    
#     fig, axs = plt.subplots (1,7, figsize=(27,20))  
#     if i>0:   #added this
#         axs[0].imshow(zcheck[i-1],cmap="bone")
#     else:
#         axs[0].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone") #empty 
#     axs[0].set_title(f"Slice {i-1} zcheck")    #change i 
#     axs[1].imshow(fatmask_R3,cmap="bone")
#     axs[1].set_title(f"Slice {i+1} raw threshold")
    
#     if i<(j-1): #if not last slice 
#         axs[2].imshow(zcheck[i+1],cmap="bone") 
#     else:
#         axs[2].imshow(np.zeros([Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]]),cmap="bone") #empty       
#     axs[2].set_title(f"Slice {i+2} zcheck")
    
#     axs[3].imshow(im_ff) #don't need check1
#     axs[3].set_title(f"Slice {i+1} SOI with Z-connections")
#     axs[4].imshow(nonZ,cmap="bone")
#     axs[4].set_title(f"Slice {i+1} nonZ")
    
#     axs[5].imshow(nonZ_keep,cmap="bone")
#     axs[5].set_title(f"Slice {i+1} nonZ_keep")
#     axs[6].imshow(final_fat_mask,cmap="bone")
#     axs[6].set_title(f"Slice {i+1} final_fat_mask")
    
#     print(f"Slice {i+1} final sum={np.sum(final_fat>0)}")
  
    return final_fat_mask, final_fat

In [98]:
# def check1(im,to_overlay,x):
#         overlay = np.ma.masked_where(to_overlay == 0, to_overlay)
#         axs[x].imshow(im, cmap="bone")
#         axs[x].imshow(overlay, cmap="hsv", vmin=0, vmax=1)

In [99]:
def apply_R3_zcheck(th,zcheck):
    final_fat_mask=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    final_fat=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    
    for i in range(j):
        final_fat_mask[i], final_fat[i]=R3_zcheck(i,th,zcheck) #tried with th[i] but diff results - images still only last image 
#         print (np.sum(final_fat_mask[i]>0))
    return  final_fat_mask, final_fat


In [100]:
def Th2finefat_z(fatfinal,th,fatfinal_mask): #z-checked
    fatseg1vol_mask_R3=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg2_refinedvol_mask_R3=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
    fatseg2_refinedvol_R3=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

    for i in range(j):
        fatseg1vol_mask_R3[i]=fatfinal[i]>th[i] #diff from R1 cus here need to apply to finalfat AFTER FINAL (R3) z-connectivity check
        fatseg2_refinedvol_mask_R3[i]= (fatseg1vol_mask_R3[i]!=fatfinal_mask[i])
        fatseg2_refinedvol_R3[i]=fatseg2_refinedvol_mask_R3[i]*Ivol8c_roi[i] 
#         print (np.sum(fatseg2_refinedvol_R3[i]>0))
    return fatseg1vol_mask_R3,fatseg2_refinedvol_mask_R3,fatseg2_refinedvol_R3
# # revise this if needed

In [101]:
#Original

#apply final zcheck
# final_fat_mask_OG, final_fat_OG=apply_R3_zcheck(thresholds2_R2_OG_rev,fatsegfinalvol_mask_R1)

#get refined fat
# fatseg1vol_mask_R3_OG,fatseg2_refinedvol_mask_R3_OG,fatseg2_refinedvol_R3_OG=Th2finefat_z(final_fat_OG,thresholds_R2_OG_rev,final_fat_mask_OG)
#stackimages(fatseg1vol_mask_R3)
# stackimages(fatseg2_refinedvol_mask_R3_OG)



In [102]:
#Original Revised

#apply final z-check
# final_fat_mask_rev, final_fat_rev=apply_R3_zcheck(thresholds2_R2_OG_rev,zcheck_Th2R2_rev)

# get refined fat 
# fatseg1vol_mask_R3_rev,fatseg2_refinedvol_mask_R3_rev,fatseg2_refinedvol_R3_rev=Th2finefat_z(final_fat_rev,thresholds_R2_OG_rev,final_fat_mask_rev)
#stackimages(fatseg1vol_mask_R3_rev)
# stackimages(fatseg2_refinedvol_mask_R3_rev)

In [103]:
#Kenneth's  

#final zcheck
# final_fat_mask_ken, final_fat_ken=apply_R3_zcheck(thresholds2_R2_ken,zcheck_Th2R2_ken)


#get refined fat
# fatseg1vol_mask_R3_ken,fatseg2_refinedvol_mask_R3_ken,fatseg2_refinedvol_R3_ken=Th2finefat_z(final_fat_ken,thresholds_R2_ken,final_fat_mask_ken)

#stackimages(fatseg1vol_mask_R3_ken)
#stackimages(fatseg2_refinedvol_mask_R3_ken)
# stackimages(fatseg2_refinedvol_R3_ken)

### Final Product

In [104]:
def check2(im, roi,x):    
    overlay = np.ma.masked_where(roi == 0, roi)
    axs[x].imshow(im, cmap="bone")
    axs[x].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)

def fats(thigh, fat2, fat3,x):    
    overlay = np.ma.masked_where(fat2 == 0, fat2)
    overlay2= np.ma.masked_where(fat3 == 0, fat3)
    axs[x].imshow(thigh, cmap="bone")
    axs[x].imshow(overlay, cmap="autumn", vmin=0, vmax=1)
    axs[x].imshow(overlay2, cmap="bwr", vmin=0, vmax=1, alpha=1)

def fats2(thigh, fat,x):
    overlay = np.ma.masked_where(fat == 0, fat)
    axs[x].imshow(thigh, cmap="bone")
    axs[x].imshow(overlay, cmap="autumn", vmin=0, vmax=1)
    
# for i in range(j):   
#     fig, axs = plt.subplots (1, 5, figsize=(17,55))
#     axs[0].set_title(f"slice {i+1}")
#     axs[0].imshow(Ivol8[i], cmap='bone')
#     axs[1].imshow(Ivol8c_roi[i], cmap='bone')
#     check2(Ivol8c_roi[i], roimaskvol[i],2)
#     fats(Ivol8c_roi[i], fatseg1vol_mask_R3_ken[i], fatseg2_refinedvol_mask_R3_ken[i],3)
#     axs[3].set_title("with Z connectivity")
#     axs[4].imshow(final_fat_ken[i], cmap='bone')
#     axs[4].set_title("with Z connectivity")
           
    


In [105]:
#og code
# for i in range(j): 
#     fig, axs = plt.subplots (1, 5, figsize=(17,55)) #need this IN the for loop
#     axs[0].imshow(Ivol8c_roi[i], cmap='bone')
#     axs[0].set_title(f"Slice {i+1} Ivol8c_roi")
#     fats(Ivol8c_roi[i], fatseg1vol_mask_R1[i], fatseg2_refinedvol_mask[i],1)
#     axs[1].set_title("WITHOUT Z connections")
#     axs[2].imshow(fatsegfinalvol_R1[i], cmap='bone')
#     axs[2].set_title("WITHOUT Z connections")

#     fats(Ivol8c_roi[i], fatseg1vol_mask_R3_ken[i], fatseg2_refinedvol_mask_R3_ken[i],3)
#     axs[3].set_title("WITH Z connections")
#     axs[4].imshow(final_fat_ken[i], cmap='bone')
#     axs[4].set_title("WITH Z connections")



## Fat/Muscle Data

### Calculations

In [106]:
def calc2(fat1,fat2,fat3,subcfat):
    MuscFatAreaPix_=[]
    MuscFatAreaMM_=[]
    MuscFatVolMM_=[]
    FatAreaPix1_slice=[]
    FatAreaMM1_slice=[]
    MuscAreaPix1_slice=[]
    MuscAreaMM1_slice=[]
    FatPerc1_slice=[]
    FatVolMM1_slice=[]
    MuscVolMM1_slice=[]
    FatAreaPix2_slice=[]
    FatAreaMM2_slice=[]
    MuscAreaPix2_slice=[]
    MuscAreaMM2_slice=[]
    FatPerc2_slice=[]
    FatVolMM2_slice=[]
    MuscVolMM2_slice=[]
    FatSegI_1_slice=[]
    FatSegI_2_slice=[]
    cfactor_slice=[]
    FatVolCombined_slice=[]
    FatVolMM2_C_slice=[]
    FatVolCombined_C_slice=[] #list for fat volumes from Th2 not corrected-will add together to get volume of all slices
    FatPerc2_C_slice=[]
    FatPercCombined_C_slice=[]
    FatAreaPix3_slice=[]
    FatAreaMM3_slice=[]
    MuscAreaPix3_slice=[]
    MuscAreaMM3_slice=[]
    FatPerc3_slice=[]
    FatVolMM3_slice=[]
    MuscVolMM3_slice=[]
    Subcfat_slice=[]
    FatVolCombined_C_all=[]
    MuscVolCombined_all=[]
    im_spacing = image.GetSpacing() #get spacing for original image - for calculations later
    ###SUBCFATMASK

    ###SUBCFATMASK
    for i in range(j):
        MuscFatAreaPix_.append([])
        MuscFatAreaMM_.append([])
        MuscFatVolMM_.append([])
        FatAreaPix1_slice.append([])
        FatAreaMM1_slice.append([])
        MuscAreaPix1_slice.append([])
        MuscAreaMM1_slice.append([])
        FatPerc1_slice.append([])
        FatVolMM1_slice.append([])
        MuscVolMM1_slice.append([])
        FatAreaPix2_slice.append([])
        FatAreaMM2_slice.append([])
        MuscAreaPix2_slice.append([])
        MuscAreaMM2_slice.append([])
        FatPerc2_slice.append([])
        FatPercCombined_C_slice.append([]) #added
        FatVolMM2_slice.append([])
        MuscVolMM2_slice.append([])
        FatSegI_1_slice.append([])
        FatSegI_2_slice.append([])
        cfactor_slice.append([])
        FatVolCombined_slice.append([])
        FatVolMM2_C_slice.append([])
        FatVolCombined_C_slice.append([])
        FatPerc2_C_slice.append([])
        FatAreaPix3_slice.append([])
        FatAreaMM3_slice.append([])
        MuscAreaPix3_slice.append([])
        MuscAreaMM3_slice.append([])
        FatPerc3_slice.append([])
        FatVolMM3_slice.append([])
        MuscVolMM3_slice.append([])
        Subcfat_slice.append([])
        FatVolCombined_C_all.append([]) #use for now to check b/w slices
        MuscVolCombined_all.append([])
    #MUSCLE + FAT AREA
        MuscFatAreaPix=np.sum(roivol[i]>0)   #not fat here is NOT corrected
        MuscFatAreaMM=MuscFatAreaPix*im_spacing[0]*im_spacing[1]
        MuscFatVolMM=MuscFatAreaMM*im_spacing[2]
    #THRESHOLD 1 FAT
        muscleroi1=roivol[i]-fat1[i]
        #plt.imshow(muscleroi1,cmap="bone")
        #plt.imshow(roivol[i],cmap="bone")
        #FAT AREA
        FatAreaPix1=np.sum(fat1[i]>0)
        FatAreaMM1=FatAreaPix1*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        MuscAreaPix1=np.sum(muscleroi1>0)
        MuscAreaMM1=MuscAreaPix1*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        FatPerc1=FatAreaPix1*100/MuscFatAreaPix #adding 2 dashes rounds it DOWN to whole number
        #VOLUME OF EACH SLICE
        FatVolMM1=FatAreaMM1*im_spacing[2] #multiply by z -slice thickness
        MuscVolMM1=MuscAreaMM1*im_spacing[2]

#THRESHOLD 2 FAT
        #NOT including threshold 1 fat----------------------------------------------
        muscleroi2=roivol[i]-fat2[i]
        #FAT AREA
        FatAreaPix2=np.sum(fat2[i]>0)
        FatAreaMM2=FatAreaPix2*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        MuscAreaPix2=np.sum(muscleroi2>0)
        MuscAreaMM2=MuscAreaPix2*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        FatPerc2=FatAreaPix2*100/MuscFatAreaPix
        #VOLUME OF EACH SLICE
        FatVolMM2=FatAreaMM2*im_spacing[2]
        MuscVolMM2=MuscAreaMM2*im_spacing[2]
        #FAT CORRECTION
        muscleroi3=roivol[i]-fat3[i]
    
        #SUBCFAT
        subcfatvol=np.sum(subcfat[i]>0)*im_spacing[0]*im_spacing[1]*im_spacing[2]
        
        
        Fat1masked=np.ma.masked_where(fat1[i]==0,fat1[i]) #brighter fat
         #less bright fat
        Musclemasked=np.ma.masked_where(muscleroi3==0,muscleroi3)
        
        

        
        array=fat1[i].flatten()
        x = [i2 for i2 in array if i2>0]


        trufatI=np.percentile(x, 75, interpolation='nearest')
        trufatI_im=np.ma.masked_where(fat1[i]<trufatI, fat1[i])
        
        below75=fat3[i].copy()
        below75[below75>trufatI]=0
        Fat2masked=np.ma.masked_where(below75==0,below75)
        
        FatSegI_1=np.mean(trufatI_im)

        FatSegI_2=np.mean(Fat2masked)

        MuscleSegI=np.mean(Musclemasked)
        
        FatAreaPix1=np.sum(trufatI_im>0)
        FatAreaMM1=FatAreaPix1*im_spacing[0]*im_spacing[1]
        
        FatAreaPix2=np.sum(Fat2masked>0)
        FatAreaMM2=FatAreaPix2*im_spacing[0]*im_spacing[1]
        
        FatPerc1=FatAreaPix1*100/MuscFatAreaPix
        FatPerc2=FatAreaPix2*100/MuscFatAreaPix
        FatVolMM1=FatAreaMM1*im_spacing[2]
        FatVolMM2=FatAreaMM2*im_spacing[2]
        
        ####SUBCFAT INTENSITY


        #fig, axs = plt.subplots(j, 2, figsize = (12, 60)) #need this??

        array=(subcfat[i]*Ivol8c_roi[i]).flatten()
        x = [i2 for i2 in array if i2>0]
        subcfatI=np.percentile(x, 50, interpolation='nearest')
        
        subcfatcorrection=(np.mean(x))/subcfatI
        print(subcfatcorrection)
        subcfatvol_C=subcfatvol*subcfatcorrection
        #print(subcfatI)
        ####SUBCFAT INTENSITY
        
        cfactor = (FatSegI_2 - MuscleSegI)/(FatSegI_1 - MuscleSegI)
        #print(subcfatI, FatSegI_1)
        
        FatVolCombined=FatVolMM1+FatVolMM2 #final volume not corrected
        FatVolMM2_C=FatVolMM2*cfactor
        FatVolCombined_C=FatVolMM1+FatVolMM2_C #final volume corrected
        FatPerc2_C=FatPerc2*cfactor
        FatPercCombined_C=FatPerc1+FatPerc2_C #NEWLY ADDED need to test this!!
    #FINAL FAT using Threshold 2 ----------------------------------------------------

        #FAT AREA
        FatAreaPix3=np.sum(fat3[i]>0)
        FatAreaMM3=FatAreaPix3*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        MuscAreaPix3=np.sum(muscleroi3>0) + (1-cfactor)*(np.sum(fat2>0))
        MuscAreaMM3=MuscAreaPix3*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        FatPerc3=FatAreaPix3*100/MuscFatAreaPix #NOT corrected tho
        #VOLUME OF EACH SLICE
        FatVolMM3=FatAreaMM3*im_spacing[2] #this is same as FatVolCombined NOT corrected- use this as a check
        MuscVolMM3=MuscAreaMM3*im_spacing[2] #can use this for final muscle volume
    #table for viewing
#         final_table=tabulate([[f'Slice {i+1} Th1','',''],['Musc + Fat Area', "%.1f" % MuscFatAreaPix,'pixels'], ['Musc + Fat Area',"%.1f" % MuscFatAreaMM,'mm^2'],['Musc + Fat Vol', "%.1f" % MuscFatVolMM,'mm^3'],[], #use ['','',''] to get rid of None for DataFrame
#         ['Musc Area 1',"%.1f" % MuscAreaPix1,'pixels'],['Musc Area 1',"%.1f" % MuscAreaMM1,'mm^2'],['Musc Vol 1', "%.1f" % MuscVolMM1,'mm^3'],[],
#         ['Fat Area 1', "%.1f" % FatAreaPix1,'pixels'],  ['Fat Area 1', "%.1f" % FatAreaMM1,'mm^2'], ['Fat Perc 1', "%.1f" % FatPerc1,'%'],['Fat Vol 1', "%.1f" % FatVolMM1,'mm^3'],[],
#         [f'Slice {i+1} Th2 Refined','',''],['Musc Area 2',"%.1f" % MuscAreaPix2,'pixels'],['Musc Area 2',"%.1f" % MuscAreaMM2,'mm^2'],['Musc Vol 2', "%.1f" % MuscVolMM2,'mm^3'],[],
#         ['Fat Area 2', "%.1f" % FatAreaPix2,'pixels'],['Fat Area 2 ', "%.1f" % FatAreaMM2,'mm^2'], ['Fat Perc 2', "%.1f" % FatPerc2,'%'],['Fat Perc 2 Corrected', "%.1f" % FatPerc2_C,'%'], ['FatPercCombined_Corrected', "%.1f" % FatPercCombined_C,'%']
#         ['Fat Vol 2 Not Corrected', "%.1f" % FatVolMM2,'mm^3'],['Fat Vol 2 Corrected', "%.1f" % FatVolMM2_C,'mm^3'],[],
#         ['Th2 Fat','',''],['Musc Area 3',"%.1f" % MuscAreaPix3,'pixels'],['Musc Area 3',"%.1f" % MuscAreaMM3,'mm^2'],['Musc Vol 3', "%.1f" % MuscVolMM3,'mm^3'],[],
#         ['Fat Area 3', "%.1f" % FatAreaPix3,'pixels'],['Fat Area 3 ', "%.1f" % FatAreaMM3,'mm^2'],['Fat Perc 3', "%.1f" % FatPerc3,'%'],['Fat Vol 3', "%.1f" % FatVolMM3,'mm^3'],[],
#         ['Fat Vol 1+2 (Not Corrected)', "%.1f" % FatVolCombined,'MM^3'],['Fat Vol 1+2c (Corrected)', "%.1f" % FatVolCombined_C,'MM^3'],[],
#         ['Th2 Fat Correction','',''],['Fat 1 Intensity', "%.1f" %FatSegI_1,'pixels'],['Fat 2 Intensity', "%.1f" %FatSegI_2,'mm^2'],['Fat Correction Factor', "%.1f" % cfactor],[]],
#         headers=[f'File {files} Slice {i+1} Data', 'Value','Units'])
        #if export final_table to excel - each sheet has info for ONE slice only
#         print(final_table)
#         MuscFatAreaPix_[i]="%.1f" %MuscFatAreaPix #"%.1f" %" to convert to 1 decimal place
        MuscFatAreaPix_[i]=MuscFatAreaPix
        MuscFatAreaMM_[i]=MuscFatAreaMM
        MuscFatVolMM_[i]=MuscFatVolMM
        FatAreaPix1_slice[i]=FatAreaPix1
        FatAreaMM1_slice[i]=FatAreaMM1
        MuscAreaPix1_slice[i]=   MuscAreaPix1
        MuscAreaMM1_slice[i]=MuscAreaMM1
        FatPerc1_slice[i]=FatPerc1
        FatVolMM1_slice[i]=FatVolMM1
        MuscVolMM1_slice[i]=MuscVolMM1
        FatAreaPix2_slice[i]=FatAreaPix2
        FatAreaMM2_slice[i]=FatAreaMM2
        MuscAreaPix2_slice[i]=MuscAreaPix2
        MuscAreaMM2_slice[i]=MuscAreaMM2
        FatPerc2_slice[i]=FatPerc2
        FatPercCombined_C_slice[i]=FatPercCombined_C #added
        FatVolMM2_slice[i]=FatVolMM2
        MuscVolMM2_slice[i]=MuscVolMM2
        FatSegI_1_slice[i]=FatSegI_1
        FatSegI_2_slice[i]=FatSegI_2
        cfactor_slice[i]=cfactor
        FatVolCombined_slice[i]=FatVolCombined
        FatVolMM2_C_slice[i]=FatVolMM2_C
        FatVolCombined_C_slice[i]=FatVolCombined_C #add all fat corrected to list
        FatPerc2_C_slice[i]=FatPerc2_C
        FatPercCombined_C_slice[i]=FatPercCombined_C
        FatAreaPix3_slice[i]=FatAreaPix3
        FatAreaMM3_slice[i]=FatAreaMM3
        MuscAreaPix3_slice[i]=MuscAreaPix3
        MuscAreaMM3_slice[i]=MuscAreaMM3
        FatPerc3_slice[i]=FatPerc3
        FatVolMM3_slice[i]=FatVolMM3
        MuscVolMM3_slice[i]=MuscVolMM3 #not corrected
        Subcfat_slice[i]=subcfatvol_C
        FatVolCombined_C_all[i]=sum(FatVolCombined_C_slice) #can't do FatVolCombined_C cus this is an ACTUAL value later
        MuscVolCombined_all[i]=sum(MuscVolMM3_slice)
        
        
    slice_num=[]
    slice_num=list(range(1,j+1)) #from 1-15
    #table for ALL data
#     all_data = {'ID': files,'Slice': slice_num,'MuscFatAreaPix':MuscFatAreaPix_, 'MuscFatAreaMM':MuscFatAreaMM_, 'MuscFatVolMM': MuscFatVolMM_,
#             'Th1_FatAreaPix':FatAreaPix1_slice,'Th1_FatAreaMM':FatAreaMM1_slice,'Th1_MuscAreaPix':MuscAreaPix1_slice,'Th1_MuscAreaMM':MuscAreaMM1_slice,
#             'Th1_FatPerc':FatPerc1_slice,'Th1_FatVolMM':FatVolMM1_slice,'Th1_MuscVolMM':MuscVolMM1_slice,
#             'Th2_RefFatAreaPix':FatAreaPix2_slice,'Th2Ref_FatAreaMM':FatAreaMM2_slice,'Th2Ref_MuscAreaPix':MuscAreaPix2_slice,'Th2Ref_MuscAreaMM':MuscAreaMM2_slice,
#             'Th2_RefFatPerc_NOTc':FatPerc2_slice,'Th2Ref_FatVolMM_NOTc':FatVolMM2_slice,'Th2Ref_MuscVolMM':MuscVolMM2_slice,
#             'Th1_FatI':FatSegI_1_slice,'Th2Ref_FatI':FatSegI_2_slice,
#             'Th2Ref_cfactor':cfactor_slice,'Th1Th2Ref_FatVol_NOTc':FatVolCombined_slice,'Th2Ref_FatVolMM_c':FatVolMM2_C_slice,
#             'Th1_Th2Ref_FatVol_c':FatVolCombined_C_slice,'Th2Ref_FatPerc_c':FatPerc2_C_slice,'Th1Th2Ref_FatPerc_c':FatPercCombined_C_slice,
#             #these area for comparisons with above - these include Th2 refined but NOT corrected:
#             'Th2_FatAreaPix_NOTc':FatAreaPix3_slice,'Th2_FatAreamm^2_NOTc':FatAreaMM3_slice,'Th2_MuscAreaPix':MuscAreaPix3_slice,
#             'Th2_MuscAreaMM':MuscAreaMM3_slice,'Th2_FatPerc_NOTc':FatPerc3_slice,
#             'Th2_FatVolMM_NOTc':FatVolMM3_slice,'Th2_MuscVolMM':MuscVolMM3_slice,
#             'TOTAL_FatVol_c':FatVolCombined_C_all,'TOTAL_MuscVol':MuscVolCombined_all} #added
#     all_data_table = pd.DataFrame(all_data, columns = ['ID','Slice','MuscFatAreaPix','MuscFatAreaMM', 'MuscFatVolMM',
#                                                     'Th1_FatAreaPix','Th1_FatAreaMM','Th1_MuscAreaPix','Th1_MuscAreaMM','Th1_FatPerc','Th1_FatVolMM','Th1_MuscVolMM',
#                                                     'Th2_RefFatAreaPix','Th2Ref_FatAreaMM','Th2Ref_MuscAreaPix','Th2Ref_MuscAreaMM','Th2_RefFatPerc_NOTc','Th2Ref_FatVolMM_NOTc','Th2Ref_FatVolMM',
#                                                     'Th1_FatI','Th2Ref_FatI','Th2Ref_cfactor','Th1Th2Ref_FatVol_NOTc','Th2Ref_FatVolMM_c','Th1_Th2Ref_FatVol_c','Th2Ref_FatPerc_c','Th1Th2Ref_FatPerc_c',
#                                                     #these area for comparisons with above - these include Th2 refined but NOT corrected:
#                                                     'Th2_FatAreaPix_NOTc','Th2_FatAreamm^2_NOTc','Th2_MuscAreaPix','Th2_MuscAreaMM','Th2_FatPerc_NOTc','Th2_FatVolMM_NOTc','Th2_MuscVolMM_NOTc','TOTAL_FatVol_c','TOTAL_MuscVol_c'])
    #table with data of interest
    
    
    data_oi = {'ID':files,'Slice': slice_num,'FatVol_c':FatVolCombined_C_slice,'FatPerc_c':FatPercCombined_C_slice,
               'MuscAreaMM':MuscAreaMM3_slice,'MuscVolMM':MuscVolMM3_slice,'TOTAL_FatVol_c':FatVolCombined_C_all,'TOTAL_MuscVol':MuscVolCombined_all} #added
    data_oi_table = pd.DataFrame(data_oi, columns = ['ID','Slice','FatVol_c','FatPerc_c','MuscAreaMM','MuscVolMM','TOTAL_FatVol_c','TOTAL_MuscVol'])
    print(sum(FatVolCombined_C_slice))
#     print(all_data_table) #this is to export to excel, to view all data properly print final_table
   # return  data_oi_table #all_data_table

#############################
    #---for Kenneth's output-----:
    Total_FatVol=sum(FatVolCombined_C_slice)
    Total_MuscVol=sum(MuscVolMM3_slice)
    Total_MuscFatVol=sum(MuscFatVolMM_)
    Total_Subcfatvol=sum(Subcfat_slice)
    
    Total_FatPerc=Total_FatVol/Total_MuscFatVol #can't make above vars lists cus can't divide lists
    Total_FatVol_l=[Total_FatVol] #need to put in list for df below
    Total_MuscVol_l=[Total_MuscVol]
    Total_FatPerc_l=[Total_FatPerc]
    Total_Subcfatvol_1=[Total_Subcfatvol]

    return Total_FatVol_l, Total_MuscVol_l, Total_FatPerc_l,data_oi_table, Total_Subcfatvol_1

# if DON'T want 2nd threshold fat from R1 + R2
## muscleroi=muscleroi2=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
#problem - printing float separately works (354.0) but not in table
def calc2(fat3):
    MuscFatAreaPix_=[]
    MuscFatAreaMM_=[]
    MuscFatVolMM_=[]
    FatAreaPix3_slice=[]
    FatAreaMM3_slice=[]
    MuscAreaPix3_slice=[]
    MuscAreaMM3_slice=[]
    FatPerc3_slice=[]
    FatVolMM3_slice=[]
    MuscVolMM3_slice=[]
    FatVol_all=[] #change name
    MuscVol_all=[]
    im_spacing = image.GetSpacing() #get spacing for original image - for calculations later
    for i in range(j):
        MuscFatAreaPix_.append([])
        MuscFatAreaMM_.append([])
        MuscFatVolMM_.append([])
        FatAreaPix3_slice.append([])
        FatAreaMM3_slice.append([])
        MuscAreaPix3_slice.append([])
        MuscAreaMM3_slice.append([])
        FatPerc3_slice.append([])
        FatVolMM3_slice.append([])
        MuscVolMM3_slice.append([])
        FatVol_all.append([])
        MuscVol_all.append([])
    #MUSCLE + FAT AREA
        MuscFatAreaPix=np.sum(roivol[i]>0)   #not fat here is NOT corrected
        MuscFatAreaMM=MuscFatAreaPix*im_spacing[0]*im_spacing[1]
        MuscFatVolMM=MuscFatAreaMM*im_spacing[2]
    #FINAL FAT using Threshold ONE from R2 with Z-connections ----------------------------------------------------
        muscleroi3=roivol[i]-fat3[i]
        #FAT AREA
        FatAreaPix3=np.sum(fat3[i]>0)
        FatAreaMM3=FatAreaPix3*im_spacing[0]*im_spacing[1]
        #MUSCLE AREA
        MuscAreaPix3=np.sum(muscleroi3>0)
        MuscAreaMM3=MuscAreaPix3*im_spacing[0]*im_spacing[1]
        #FAT PERCENTAGE
        FatPerc3=FatAreaPix3*100/MuscFatAreaPix #NOT corrected tho
        #VOLUME OF EACH SLICE
        FatVolMM3=FatAreaMM3*im_spacing[2] #this is same as FatVolCombined NOT corrected- use this as a check
        MuscVolMM3=MuscAreaMM3*im_spacing[2] #can use this for final muscle volume
        MuscFatAreaPix_[i]=MuscFatAreaPix
        MuscFatAreaMM_[i]=MuscFatAreaMM
        MuscFatVolMM_[i]=MuscFatVolMM
        FatAreaPix3_slice[i]=FatAreaPix3
        FatAreaMM3_slice[i]=FatAreaMM3
        MuscAreaPix3_slice[i]=MuscAreaPix3
        MuscAreaMM3_slice[i]=MuscAreaMM3
        FatPerc3_slice[i]=FatPerc3
        FatVolMM3_slice[i]=FatVolMM3
        MuscVolMM3_slice[i]=MuscVolMM3 #not corrected
        FatVol_all[i]=sum(FatVolMM3_slice) #changed from OG
        MuscVol_all[i]=sum(MuscVolMM3_slice)
    slice_num=[]
    slice_num=list(range(1,j+1)) #from 1-15
    #table with data of interest -use this to compare with output vars below
    data_oi = {'ID':files,'Slice': slice_num,'FatVol':FatVolMM3_slice,'FatPerc_c':FatPerc3_slice,
               'MuscAreaMM':MuscAreaMM3_slice,'MuscVolMM':MuscVolMM3_slice,'TOTAL_FatVol_c':FatVol_all,'TOTAL_MuscVol':MuscVol_all} #added
    data_oi_table = pd.DataFrame(data_oi, columns = ['ID','Slice','FatVol_c','FatPerc_c','MuscAreaMM','MuscVolMM','TOTAL_FatVol_c','TOTAL_MuscVol'])
    print(f'{files} total fatvol= {sum(FatVolMM3_slice)}')
    #---for Kenneth's output-----:
    Total_FatVol=sum(FatVolMM3_slice) #take sum of slice volumes in list
    Total_MuscVol=sum(MuscVolMM3_slice)
    Total_MuscFatVol=sum(MuscFatVolMM_)
    Total_FatPerc=Total_FatVol/Total_MuscFatVol #can't make above vars lists cus can't divide lists
    Total_FatVol_l=[Total_FatVol] #need to put in list for df below
    Total_MuscVol_l=[Total_MuscVol]
    Total_FatPerc_l=[Total_FatPerc]
#     Total_FatVol_df=pd.DataFrame({"ID":files,"Total_FatVol":Total_FatVol_l}) #turn to dataframe to export to excel
#     Total_MuscVol_df=pd.DataFrame({"ID":files,"Total_MuscVol":Total_MuscVol_l})
#     Total_FatPerc_df=pd.DataFrame({"ID":files,"Total_FatPerc":Total_FatPerc_l})
#     Total_FatVol_table=pd.DataFrame(Total_FatVol_df, columns = ['ID','Total_FatVol'])
#     Total_MuscVol_table=pd.DataFrame(Total_MuscVol_df, columns = ['ID','Total_MuscVol'])
#     Total_FatPerc_table=pd.DataFrame(Total_FatPerc_df, columns = ['ID','Total_FatPerc'])
#     print(f'Total_FatVol={Total_FatVol}\nTotal_MuscVol={Total_MuscVol}\nTotal_MuscFatVol1={Total_MuscFatVol}\nTotal_FatPerc1={Total_FatPerc}')
    #incldued data_oai_table to return for CHECKING
#     return Total_FatVol_table, Total_MuscVol_table, Total_FatPerc_table,data_oi_table #all_data_table
    return Total_FatVol_l, Total_MuscVol_l, Total_FatPerc_l,data_oi_table
# all_data_table=calc2(fatseg1vol_R2_ken,fatseg2_refinedvol_R3_ken,final_fat_ken)







In [107]:
#export table to excel
# with pd.ExcelWriter('all_data_table.xlsx') as writer:  
#         all_data_table.to_excel(writer, sheet_name='all_data_table',index=False) 



In [108]:
#write to existing excel
# import pandas
# from openpyxl import load_workbook

# with pandas.ExcelWriter('Masterfile.xlsx', engine='openpyxl',mode='a') as writer: #mode=a for appending 
#     data_filtered.to_excel(writer, sheet_name='')

# OR
# t1=pd.DataFrame(t1)
# t2=pd.DataFrame(t2)
# with pd.ExcelWriter('ITSAresults.xlsx') as writer:  
#     t1.to_excel(writer, sheet_name='t1')
#     t2.to_excel(writer, sheet_name='t2')

In [109]:
def mylistdir(directory):
    """A specialized version of os.listdir() that ignores files that
    start with a leading period."""
    filelist = os.listdir(directory)
    return [x for x in filelist
            if not (x.startswith('.'))]


    
#test to see if this works in the tables error thing

## Apply on ALL

In [126]:
path="path"
# fileslist=os.listdir(path)
# fileslist.remove('.DS_Store')
# print(fileslist)

fileslist=sorted(mylistdir(path))
print(fileslist)

ID_count=0
table=[]
Th1R1_ID=[]
Th2R1_ID=[]
Th1R2_ID=[]
Th2R2_ID=[]
success_IDs=[]
error_IDs=[]

#added for Kenneth's outputs:
Total_FatVol_ID=[]
Total_MuscVol_ID=[]
Total_FatPerc_ID=[]
Total_SubcFat_ID=[]

for files in fileslist: #files = ID number 
    print(files)
    try:
        image_path = os.path.join(path,files)

        slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(image_path)
        image = sitk.ReadImage(slice_filenames)

        Ivol8,j,plotx=getimages(image) #Get all images
        #stackimages(Ivol8)

        #---------IH CORRECTION--------#
        IH1vol_mask=get_IH1vol_mask() #get mask for IH correction
        preIvol8c_roi=IH1() #IH correction using mask 
        IH2vol_mask=get_IH2vol_mask() #get mask from corrected image
        IH3vol=IH2() #IH correction using new mask 
        Ivol8c_roi=IH_roi() #IH correction for roi (no mask used)

        IH3vol_mask=subcfat(IH3vol) #??
        #stackimages(IH3vol_mask) 
        Ivol8c_subcfat=IH3()

        #-----------REMOVE VESSELS + SNAKES---------#
        curvature_flow=apply_CurvatureFilter()
        Ivol8c_c_s=enhance_sharpness()
        subcfatvol=get_subcfatvol()
        bilateral_filter=BilateralFilter(Ivol8c_c_s)

        ##**
        thighpremask=get_thighpremask(Ivol8c_subcfat)
        thighmask, skin_fat=thigh_mask()
        subcfatvol=subcfatvol*thighmask
        subcfatvol_augmented=subcfatvol+skin_fat
        th, subcfatvol_augmented = cv2.threshold(subcfatvol_augmented, 0, 1, cv2.THRESH_BINARY)
        ##**

        no_skin=thigh_minus_skin()
        #stackimages(no_skin)
        t, t2=darkpieces()
        #stackimages(t)

        #subcfatvol=subcfat(Ivol8c_subcfat) #??do we need this
        #stackimages(subcfatvol)#and this??
        otsu, vessels_removed_mask, vessels_present=remove_vessels(Ivol8c_c_s) #remove vessels 
        # stackimages(vessels_removed_mask)

        pre_vessels_mask=vessels_removed_mask-vessels_present #draw holes that were removed #??why no need for empty array
        #stackimages(pre_vessels_mask) 

        vessels_mask=get_vessels() #area check- function of radius and length to isolate the circles (represents vessels)
        #stackimages(vessels_mask)

        whites=get_whites() #generate whites mask to guide snake away from thigh border&vessels (which are dark)- snake is attracted to dark lines
        #stackimages(whites)

        maskoutlinevol, contoursvol=get_contours(t2)  #initiate snake with an initial contour (of muscle mask) 
        #stackimages(maskoutlinevol)
        snakeinitiator_draw, hull_demonstration=convex_hull(contoursvol) #get convex hull of muscle mask     
        #show_convexhull()

        hullmask=floodfillall(snakeinitiator_draw)

        t3=get_t3() #connect fragments of fascia with hull to generate new hull
        #stackimages(t3)
        maskoutlinevol, contoursvol=get_contours(t3) #??is this maskoutlinevol meant to replace the one above??
        #stackimages(maskoutlinevol) 

        snakeinitiator_draw, hull_demonstration=convex_hull(contoursvol) #??
        #show_convexhull()

        hullmask_premerge= floodfillall(snakeinitiator_draw) #floodfill before combining hulls #??why tho - is the floodfill the checking?
        #stackimages(hullmask_premerge)

        merged_hulls=merge_hulls() #add hulls to find overlap
        #stackimages(merged_hulls)
        merged, fullcontoursvol=get_contours(merged_hulls) #get contours
        #stackimages(merged)

        hull2vol=compatible_coordlist(fullcontoursvol) #get coordinates of contours??
        #stackimages(merged)
        snake_im=get_snakeim()  #apply white mask to filtered?? image - snake will operate on this image 
        #stackimages(snake_im)

        snakevol_p=primary_snake() #get list of primary snake coordinates??

        snakemaskvol_p=snaketomask(snakevol_p) #use snake coordinates to get snake mask??
        #stackimages(snakemaskvol_p)

        snakevol_s=secondary_snake() #get list of secondary snake coordinates??
        snakemaskvol_s=snaketomask(snakevol_s) #use snake coordinates to get snake mask??
        #stackimages(snakemaskvol_s)

        add=snakemaskvol_p+snakemaskvol_s #add primary + secondary snake 
        ret, add = cv2.threshold(add,0,1,cv2.THRESH_BINARY) #get binary threshold 
        #stackimages(add)

        drawing3vol, fullcontoursvol=get_contours(add)
        hull4vol=compatible_coordlist(fullcontoursvol)
        #stackimages(drawing3vol)

        snakevol3=snake_finetune()  #??

        snakemaskvol=snaketomask(snakevol3)
        #stackimages(snakemaskvol)
        musclemaskvol2=snakemaskvol+t2 #add snake+muscle roi to account for missed pieces
        ret, musclemaskvol2 = cv2.threshold(musclemaskvol2,0,1,cv2.THRESH_BINARY)

        snakemask_premerge=snake_postprocessing() 
        #stackimages(snakemask_premerge+musclemaskvol2)

        snake_merge=get_snakeoverlap() #keep overlapping snakes 
        musclemask=getmusclemaskpt2()


        #------------BONE AND DEFECT REMOVAL-----------------#
        cortical=corticaloutline()
        Ivol8c_boneprep_mask = get_bonemask()
        #stackimages(Ivol8c_boneprep_mask)
        bonemwvol=find_potentialbone(Ivol8c_boneprep_mask) #initiates bone search by finding contours similar to bone
        bonemvol=floodfillall(bonemwvol) #floodfill bone   
        bonemvol=bonemvol*merged_hulls
        #stackimages(bonemvol)    
        bonemvols=finalize_bonemask(bonemvol) ##label bone objects and keep the bone based off of shape
        #stackimages(bonemvols)


        ######################
        #try the snake with just enhanced sharpness; need to convert from im to array with PIL
        #next, could try to set a floor value so that there is decreased competition from the contour of the muscle

        #make an experimentation file and a separate file for teh master copy

        Ivol8c_c_s2=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

        for i in range(j):
            f=curvature_flow[i].copy()
            f=Image.fromarray(f, mode=None)
            enhancer = ImageEnhance.Sharpness(f)


            factor =  12.0

            Ivol8c_c_s2[i]=enhancer.enhance(factor)

        fg=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')
        Ivol8c_c_s2_b=np.zeros([Ivol8c_roi.shape[0], Ivol8c_roi.shape[1], Ivol8c_roi.shape[2]], dtype='uint8')

        for i in range(j):
            Ivol8c_c_s2_b[i] = cv2.bilateralFilter(Ivol8c_c_s2[i],20,35,35)

        u=subcfat(Ivol8c_c_s2_b)

        for i in range(j):
            fg[i]= cv2.erode(u[i],np.ones((2,2),np.uint8),iterations = 1) #prep for watershed (sure bone) and rule out connectivity with muscle fat

        #be wary of the object removal in the object removal in the subfat removal function. should make a separate func for bone identification. lazy mode rn.

        #stackimages(u)
        #stackimages(fg)


        ######################

        fibulamarrow=recoverfibula()
        recoveredmarrow=recoverfibulacortical()


        th, bonemvols = cv2.threshold(bonemvols+recoveredmarrow, 0, 1, cv2.THRESH_BINARY)



        #stackimages(bonemvols)
        corticalt=merge_bones()
        bonemwvol=find_potentialbone(corticalt)
        #stackimages(bonemwvol)
        bonemvol=floodfillall(bonemwvol)
        #stackimages(bonemvol)
        bonemvols=finalize_bonemask_cortical(bonemvol)
        bonemvolz=cleanup_bone(bonemvols)
        im=watershed_bone()
        #stackimages(im)


        defect_final=get_defects()

        snake_defectremoved=musclemaskvol2-defect_final

        ###################
        for i in range(j):
            snake_defectremoved[i]=cv2.morphologyEx(snake_defectremoved[i], cv2.MORPH_OPEN, np.ones((13,13),np.uint8))
            snake_defectremoved[i]= (morphology.remove_small_objects(label(snake_defectremoved[i]),min_size=1800, connectivity=1))
            th, snake_defectremoved[i] = cv2.threshold(snake_defectremoved[i], 0, 1, cv2.THRESH_BINARY)


        for i in range(j):
            snake_defectremoved[i]=cv2.morphologyEx(snake_defectremoved[i], cv2.MORPH_OPEN, np.ones((43,43),np.uint8))

        testoverlay=0
        for i in range(j):
            testoverlay+=snake_defectremoved[i]

        #generate full contour list #COPIED
        fullcontoursvol=[]

        merged = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)

        for i in range(j):
            fullcontoursvol.append([])
            fullcontoursvol[i]= cv2.findContours(np.uint8(testoverlay>8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for i in range(j):
            for i2 in range(len(fullcontoursvol[i])):
                color = (255, 0, 0) # blue - color for convex hull
                cv2.drawContours(merged[i], fullcontoursvol[i][0],  -1, (1,0,0), 1)   


        hull2vol=[]

        for i in range(j):
            hull2vol.append([])
            for i2 in range(len(fullcontoursvol[i][0][0])):
                hull2vol[i].append(fullcontoursvol[i][0][0][i2][0])


            hull2vol[i]=np.array(hull2vol[i])
            hull2vol[i]=hull2vol[i].astype(float)

        #fine tune

        snakevol3=[]


        for i in range(j):
            snakevol3.append([])
            snakevol3[i]= active_contour(snake_im[i], hull2vol[i],alpha=0.1, beta=1, w_line=-1.5, w_edge=2,gamma=0.5, convergence=0.8)



        drawing3vol = np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)
        vol=[]

        for i in range(j):
            vol.append([])
            for i2 in range(len(snakevol3[i])):
                vol[i].append([])
                vol[i][i2].append(snakevol3[i][i2])

            vol[i]=np.rint(vol[i]).astype(int)

            cv2.drawContours(drawing3vol[i], vol[i],  -1, (1,0,0), 1)   


        drawsnakevol= np.zeros((Ivol8c_subcfat.shape[0], Ivol8c_subcfat.shape[1], Ivol8c_subcfat.shape[2]), np.uint8)

        for i in range(j):
            cv2.polylines(drawsnakevol[i], [vol[i]], isClosed=True, color = (1, 0, 0) , thickness=1) 


        snakemaskvol=drawsnakevol.copy()

        h, w = subcfatvol[i].shape[:2]

        for i in range (j):
            mask = np.zeros((h+2, w+2), np.uint8)
            (x,y),radius = cv2.minEnclosingCircle(vol[i]) #(x, y) coordinates of the centre of the "circle"
            cv2.floodFill(snakemaskvol[i], mask, (round(x),round(y)), 255)


        ret, snakemaskvol = cv2.threshold(snakemaskvol,0,1,cv2.THRESH_BINARY)

        snakemaskvol=snakemaskvol-defect_final


        th, snake_defectremoved[i] = cv2.threshold(snake_defectremoved[i], 0, 1, cv2.THRESH_BINARY)




        ###########################
        roimaskvol=roimask() #remove bone from muscle mask to get roi mask

        ########
        ret, roimaskvol = cv2.threshold(roimaskvol,0,1,cv2.THRESH_BINARY)
        roimaskvol=roimaskvol-recoveredmarrow
        roimaskvol[roimaskvol==255]=0
        ########

        #stackimages(roimaskvol)
        #check_roi()
        roivol=roimaskvol*Ivol8c_roi
        #stackimages(roivol)


        #----------ROUND 1: Fat segmentation---------------#
        Th1R1_ID.append([])
        Th2R1_ID.append([])
        Th1R2_ID.append([])
        Th2R2_ID.append([])

        #Th Optimization 1
        initial_th1_R1=initial_th(roivol) #get initial th for optimization loop using multi-otsu on roivol
        Th1_R1,fatseg1vol_mask_R1,fatseg1vol_R1=opt_Th1R1() #get optimized thresholds 1 for Round 1
        Th1R1_ID[ID_count]=Th1_R1 #added
        #stackimages(fatseg1vol_R1)

        #Th Optimization 2
        roi2vol=subtract_fat1(fatseg1vol_R1) #remove the first round of fat
        initial_th2_R1=initial_th(roi2vol) #get initial th for optimization loop using multi-otsu on roi2vol
        Th2_R1=opt_Th2R1() #get optimized thresholds 2 for Round 1
        Th2R1_ID[ID_count]=Th2_R1 #added

        fatsegfinalvol_mask_R1,fatsegfinalvol_R1=fatfinal_R1() #apply Th2 to get fat from Th1
        # stackimages(fatsegfinalvol_mask_R1)  #view R1 final fat mask
        #stackimages(fatsegfinalvol_R1) #veiw R1 final fat
        fatseg2_refinedvol_mask,fatseg2_refinedvol=Th2finefat() #get finer fat from Th2 loop not captured in Th1 loop
        #stackimages(fatseg2_refinedvol)

        #--------ROUND 2: Fat segmentation + Z connectivity Check-----#

        #Th Optimization 1
        zcheck_Th2R1=apply_th(Th2_R1) #apply th 2 from Round 1 to get images to be used for z-connectivity checking (for kenneth's)

        #OG + OG Rev
        #fatseg1vol_mask_R2_OG_rev,fatseg1vol_R2_OG_rev, thresholds_R2_OG_rev=opt_Th1R2(roivol,initial_th1_R1,fatsegfinalvol_mask_R1) #Th1 R2

        #Kenneth's
        fatseg1vol_mask_R2_ken,fatseg1vol_R2_ken, thresholds_R2_ken=opt_Th1R2(roivol,initial_th1_R1,zcheck_Th2R1) #Th1 R2
        Th1R2_ID[ID_count]=thresholds_R2_ken #added
        #Th Optimization 2

        #OG + OG Rev
        # roi2vol_R2_OG_rev=subtract_fat1(fatseg1vol_R2_OG_rev) #Remove the first round of fat
        # initial_th2_R2_OGrev=initial_th(roi2vol_R2_OG_rev) #get initial th for Th2 loop Round 2
        # fatseg2vol_R2_OG_rev,fatseg2vol_mask_R2_OG_rev, thresholds2_R2_OG_rev=opt_Th2R2(roi2vol_R2_OG_rev, initial_th2_R2_OGrev, fatsegfinalvol_mask_R1)#Th2 R2

        #Kenneth's
        roi2vol_R2_ken=subtract_fat1(fatseg1vol_R2_ken) #Remove the first round of fat
        initial_th2_R2_ken=initial_th(roi2vol_R2_ken)#get initial th for Th2 loop Round 2
        fatseg2vol_R2_ken,fatseg2vol_mask_R2_ken,thresholds2_R2_ken=opt_Th2R2(roi2vol_R2_ken, initial_th2_R2_ken, zcheck_Th2R1)#Th2 R2
        Th2R2_ID[ID_count]=thresholds2_R2_ken #added


        #-----ROUND 3: Final Z-check--------#

        #OG
        # final_fat_mask_OG, final_fat_OG=apply_R3_zcheck(thresholds2_R2_OG_rev,fatsegfinalvol_mask_R1)#apply final zcheck
        # fatseg1vol_mask_R3_OG,fatseg2_refinedvol_mask_R3_OG,fatseg2_refinedvol_R3_OG=Th2finefat_z(final_fat_OG,thresholds_R2_OG_rev,final_fat_mask_OG)#get refined fat

        #OG Rev
        # zcheck_Th2R2_rev=apply_th(thresholds2_R2_OG_rev) #apply Th2 R2 
        # final_fat_mask_rev, final_fat_rev=apply_R3_zcheck(thresholds2_R2_OG_rev,zcheck_Th2R2_rev)#apply final zcheck
        # fatseg1vol_mask_R3_rev,fatseg2_refinedvol_mask_R3_rev,fatseg2_refinedvol_R3_rev=Th2finefat_z(final_fat_rev,thresholds_R2_OG_rev,final_fat_mask_rev)#get refined fat

        #Kenneth's
        zcheck_Th2R2_ken=apply_th(thresholds2_R2_ken) #apply Th2 R2
        final_fat_mask_ken, final_fat_ken=apply_R3_zcheck(thresholds2_R2_ken,zcheck_Th2R2_ken)#apply final zcheck
        fatseg1vol_mask_R3_ken,fatseg2_refinedvol_mask_R3_ken,fatseg2_refinedvol_R3_ken=Th2finefat_z(final_fat_ken,thresholds_R2_ken,final_fat_mask_ken)#get refined fat

        print (f"ID {files} done")

##siwensv
        fatseg1vol_R3_ken_L=fatseg1vol_mask_R3_ken*Ivol8c_roi
##siwens^
        
        table.append([])
        Total_FatVol_ID.append([]) #for kenneth's output
        Total_MuscVol_ID.append([])#for kenneth's output
        Total_FatPerc_ID.append([])#for kenneth's output
        Total_SubcFat_ID.append([])
        
        snakemaskvol[snakemaskvol==255]=0
        subcfatmask= np.zeros([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')
        
        for i in range(j):
            subcfatmask[i]=((thighmask[i]-snakemaskvol[i])*Ivol8c_roi[i])>Th1R2_ID[ID_count][i]
            subcfatmask[i][subcfatmask[i]==255]=0
        
        
        Total_FatVol_ID[ID_count],Total_MuscVol_ID[ID_count],Total_FatPerc_ID[ID_count],table[ID_count], Total_SubcFat_ID[ID_count]=calc2(fatseg1vol_R3_ken_L,fatseg2_refinedvol_R3_ken,final_fat_ken, subcfatmask)

        for i in range(j): 
            fig, axs = plt.subplots (1, 4, figsize=(17,55)) #need this IN the for loop
            axs[0].imshow(Ivol8c_roi[i], cmap='bone')
            axs[0].set_title(f"{files} Slice {i+1} Ivol8c_roi")
            check2(Ivol8c_roi[i], roimaskvol[i],1)
            fats(Ivol8c_roi[i], fatseg1vol_mask_R3_ken[i], fatseg2_refinedvol_mask_R3_ken[i],2)
            axs[2].set_title("WITH Z connections")
            check2(Ivol8c_roi[i], subcfatmask[i],3)



    #     table.append([])
    #     table[ID_count]=calc2(fatseg1vol_R2_ken,fatseg2_refinedvol_R3_ken,final_fat_ken)

        ID_count+=1
        print("finished", ID_count, "/", len(fileslist) )

    except:
         print(f"Error occurred for {files}")
    #         errors_ID.append(files)
    else:
        print(f"{files} successful")#still ned checks for each fxn tho
        success_IDs.append(files)


['2.16.124.113531.4.5.129436358246560000.676719281.dcm', '2.16.124.113531.4.5.129436358251710000.676719282.dcm', '2.16.124.113531.4.5.129436358256870000.676719283.dcm', '2.16.124.113531.4.5.129436358262030000.676719284.dcm', '2.16.124.113531.4.5.129436358264060000.676719285.dcm', '2.16.124.113531.4.5.129436358268900000.676719286.dcm', '2.16.124.113531.4.5.129436358271090000.676719287.dcm', '2.16.124.113531.4.5.129436358273590000.676719288.dcm', '2.16.124.113531.4.5.129436358276400000.676719289.dcm', '2.16.124.113531.4.5.129436358279210000.676719290.dcm']
2.16.124.113531.4.5.129436358246560000.676719281.dcm
Error occurred for 2.16.124.113531.4.5.129436358246560000.676719281.dcm
2.16.124.113531.4.5.129436358251710000.676719282.dcm
Error occurred for 2.16.124.113531.4.5.129436358251710000.676719282.dcm
2.16.124.113531.4.5.129436358256870000.676719283.dcm
Error occurred for 2.16.124.113531.4.5.129436358256870000.676719283.dcm
2.16.124.113531.4.5.129436358262030000.676719284.dcm
Error occur

In [127]:
#siwen here


In [128]:
fat1=([x for x in (fatseg1vol_R3_ken_L[3].flatten()) if x>0]) #round 1 fat
fat2=([x for x in (fatseg2_refinedvol_R3_ken[3].flatten()) if x>0]) #round 2 fat
muscle=([x for x in ((roivol-final_fat_ken)[3].flatten()) if x>0])

print(np.median(fat1), np.median(fat2), np.median(muscle))

NameError: name 'fatseg1vol_R3_ken_L' is not defined

In [ ]:
print("ranges")
print("round 1:",np.amin(fat1),np.amax(fat1))
print("round 2:",np.amin(fat2),np.amax(fat2))
print("muscle:",np.amin(muscle),np.amax(muscle))
print()

print("means")
print("round 1:",np.mean(fat1))
print("round 2:",np.mean(fat2))
print("muscle:",np.mean(muscle))
print()

print("SDs")
print("round 1:",np.std(fat1))
print("round 2:",np.std(fat2))
print("muscle:",np.std(muscle))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 6))
ax[0].imshow(fatseg2_refinedvol_R3_ken[3], cmap='bone')
ax[1].imshow(fatseg2_refinedvol_R3_ken[3]>80, cmap='bone')

fig, ax = plt.subplots(1,2, figsize=(12, 6))
ax[0].imshow(fatseg2_refinedvol_R3_ken[3], cmap='bone')
ax[1].imshow((fatseg2_refinedvol_R3_ken[3]>77).astype(int), cmap='bone')


In [ ]:
plt.imshow((fatseg1vol_R3_ken_L[3]>0)+(fatseg2_refinedvol_R3_ken[3]>77))

In [ ]:
corrected=(fatseg2_refinedvol_R3_ken.copy())>0

for i in range(j):
    muscle=([x for x in ((roivol-final_fat_ken)[i].flatten()) if x>0])
    corrected[i]=(fatseg1vol_R3_ken_L[i]+(corrected[i]>(np.mean(muscle)+2*np.std(muscle))))


In [ ]:
plt.imshow((corrected[8].astype(int)+corrected[9].astype(int)+corrected[7].astype(int)))

In [ ]:
print(np.median(fat1))
belowmedian=[x for x in fat1 if x<np.median(fat1)]

print("range:",np.amin(belowmedian),np.amax(belowmedian))
print("mean:",np.mean(belowmedian))


print("stdevs:", np.std(belowmedian),np.std(fat1))

In [ ]:
print(np.std(fat1))
stackimages(fatseg1vol_R3_ken_L)


In [ ]:
# array=(subcfat[i]*Ivol8c_roi[i]).flatten()
# x = [i2 for i2 in array if i2>0]

print(np.median([x for x in (fatseg2_refinedvol_R3_ken.flatten()) if x>0]))

stackimages(fatseg2_refinedvol_R3_ken)

In [ ]:
print(np.median([x for x in ((roivol-final_fat_ken).flatten()) if x>0]))
stackimages(roivol-final_fat_ken)

success_IDs2=[] #need this for below - ensuring IDs match for table + thresholds list
ID_count=0
for i in range(len(success_IDs)):
    success_IDs2.append([])
    success_IDs2[ID_count]=success_IDs[i]
    print(success_IDs2[ID_count])
    ID_count+=1
# print(success_IDs2)

In [ ]:
beep beep error

In [ ]:
# Get ALL thresholds - check for errors
slice_num=[]
slice_num=list(range(1,j+1)) #from 1-15
all_th=[]
all_th_table=[]
final_table=[]
ID_count=0
for files in os.listdir(path):
    all_th.append([])
    all_th_table.append([])
    final_table.append([])
    all_th[ID_count]= {'Th1_R1':Th1R1_ID[ID_count],'Th2_R1':Th2R1_ID[ID_count],'Th1_R2':Th1R2_ID[ID_count],'Th2_R2':Th2R2_ID[ID_count]}  #don't need ID and slice# cus present in table[ID_count]
    all_th_table[ID_count] = pd.DataFrame(all_th[ID_count], columns = ['Th1_R1','Th2_R1','Th1_R2','Th2_R2'])
    #concatenate two tables
    final_table[ID_count]=pd.concat([table[ID_count],all_th_table[ID_count]], axis=1)
    ID_count+=1
print(final_table[0])
print(final_table[1])

In [ ]:
# WORKS!!! - add in total values for muscle + fat volumes
import pandas


count=0
for files in fileslist:
    try:
        if count==0:#if first ID
            with pd.ExcelWriter('C1 subcfat.xlsx') as writer:
                table[count].to_excel(writer, sheet_name=f'{files}',index=False)
         #write to existing excel
        else:
            with pandas.ExcelWriter('C1 subcfat.xlsx', engine='openpyxl',mode='a') as writer: #mode=a for appending
                table[count].to_excel(writer, sheet_name=f'{files}')
        count+=1
        
    except:
        print(f"Error occurred for {files}")
    else:
        print(f" {files} successful")

In [ ]:
def append_df_to_sheet(to_append,excel_path,sheetname):
    df_excel = pd.read_excel((excel_path),sheet_name=sheetname) #read specific sheet name
    result = pd.concat([df_excel,to_append], ignore_index=True) #concatenate sheet contents with new df
    return result



In [ ]:
#1
ID_count=0
store_dir="C1 subcfat.xlsx" #need r for raw string
Total_FatVol_table=[]
Total_MuscVol_table=[]
Total_FatPerc_table=[]
Total_SubcFat_table=[]
for ID in range(len(success_IDs)):
    Total_FatVol_df=pd.DataFrame({"ID":success_IDs[ID_count],"FatVol":Total_FatVol_ID[ID_count]}) #turn to dataframe to export to excel
    Total_FatPerc_df=pd.DataFrame({"ID":success_IDs[ID_count],"FatPerc":Total_FatPerc_ID[ID_count]})
    Total_MuscVol_df=pd.DataFrame({"ID":success_IDs[ID_count],"MuscVol":Total_MuscVol_ID[ID_count]})
    Total_SubcFat_df=pd.DataFrame({"ID":success_IDs[ID_count],"SubcFatVol":Total_SubcFat_ID[ID_count]})
    
    Total_FatVol_table=pd.DataFrame(Total_FatVol_df, columns = ['ID','FatVol'])
    Total_MuscVol_table=pd.DataFrame(Total_MuscVol_df, columns = ['ID','MuscVol'])
    Total_FatPerc_table=pd.DataFrame(Total_FatPerc_df, columns = ['ID','FatPerc'])
    Total_SubcFat_table=pd.DataFrame(Total_SubcFat_df, columns = ['ID','SubcFatVol'])
    
    if ID_count==0:
        with pd.ExcelWriter(store_dir) as writer:
            Total_FatVol_table.to_excel(writer, sheet_name=f'FatVol',index=False)
            Total_FatPerc_table.to_excel(writer, sheet_name=f'FatPerc',index=False)
            Total_MuscVol_table.to_excel(writer, sheet_name=f'MuscVol',index=False)
            Total_SubcFat_table.to_excel(writer, sheet_name=f'SubcFatVol',index=False)
            table[ID_count].to_excel(writer, sheet_name=f'{success_IDs[ID_count]}',index=False)
    else: #add to existing sheet
        FatVol_append=append_df_to_sheet(Total_FatVol_table,store_dir,'FatVol') #??why can't put FatVol_append[ID_count]??
        FatPerc_append=append_df_to_sheet(Total_FatPerc_table,store_dir,'FatPerc')
        MuscVol_append=append_df_to_sheet(Total_MuscVol_table,store_dir,'MuscVol')
        SubcFat_append=append_df_to_sheet(Total_SubcFat_table,store_dir,'SubcFatVol')
        with pd.ExcelWriter(store_dir) as writer:
            FatVol_append.to_excel(writer, sheet_name=f'FatVol',index=False)
            FatPerc_append.to_excel(writer, sheet_name=f'FatPerc',index=False)
            MuscVol_append.to_excel(writer, sheet_name=f'MuscVol',index=False)
            SubcFat_append.to_excel(writer, sheet_name=f'SubcFatVol',index=False)
    ID_count+=1

In [ ]:
def append_column_to_sheet(to_append1_name,to_append1,to_append2_name,to_append2, excel_path,sheetname):
    df_excel = pd.read_excel((excel_path),sheet_name=sheetname) #read specific sheet name
    df_excel[to_append1_name]=to_append1
    df_excel[to_append2_name]=to_append2
    return df_excel

In [ ]:
#2
ID_count=0  #for test retest loop + FU loop
#added columns need to be same length as column already present - so need to make list here
new_fatvol_col=[]
new_fatperc_col=[]
new_muscvol_col=[]
for ID in range(len(success_IDs)): #add new loop results into a list
    new_fatvol_col.append(Total_FatVol_ID[ID_count][0]) #[0] to take out of list
    new_fatperc_col.append(Total_FatPerc_ID[ID_count][0])
    new_muscvol_col.append(Total_MuscVol_ID[ID_count][0])
    ID_count+=1
print(new_fatvol_col)
print(new_fatperc_col)
print(new_muscvol_col)



In [ ]:
#3
# ADD new column


FatVol2=append_column_to_sheet('ID2',success_IDs,'Test-Retest',new_fatvol_col,store_dir,'FatVol')
FatPerc2=append_column_to_sheet('ID2',success_IDs,'Test-Retest',new_fatperc_col,store_dir,'FatPerc')
MuscVol2=append_column_to_sheet('ID2',success_IDs,'Test-Retest',new_muscvol_col,store_dir,'MuscVol')

store_dir="test_followup.xlsx" #need r for raw string

with pd.ExcelWriter(store_dir) as writer:
    FatVol2.to_excel(writer, sheet_name=f'FatVol',index=False)
    FatPerc2.to_excel(writer, sheet_name=f'FatPerc',index=False)
    MuscVol2.to_excel(writer, sheet_name=f'MuscVol',index=False)

In [ ]:
fig, axs = plt.subplots(j, 2, figsize=(12, j*6))
for i in range(j):

    def check2(im, roi):    
        overlay = np.ma.masked_where(roi[i] == 0, roi[i])
        axs[i,1].imshow(im[i], cmap="bone")
        
        axs[i,1].imshow(overlay, cmap="hsv", vmin=0, vmax=1, alpha=0.4)
        
    axs[i, 0].set_title(f"slice {i+1}", fontsize=12)   
    axs[i,0].imshow(Ivol8c_roi[i], cmap='bone')
    check2(Ivol8c_roi, merged_hulls)

In [ ]:
# for files in os.listdir(path): 
#     print(files)

In [ ]:
# for files in os.listdir(path):  #here!!!
# #     stackimages(fatseg1vol_R1)
#     for i in range(j): 
#         fig, axs = plt.subplots (1, 5, figsize=(17,55)) #need this IN the for loop
#         axs[0].imshow(Ivol8c_roi[i], cmap='bone')
#         axs[0].set_title(f"Slice {i+1} Ivol8c_roi")
#         fats(Ivol8c_roi[i], fatseg1vol_mask_R1[i], fatseg2_refinedvol_mask[i],1)
#         axs[1].set_title("WITHOUT Z connections")
#         axs[2].imshow(fatsegfinalvol_R1[i], cmap='bone')
#         axs[2].set_title("WITHOUT Z connections")

#         fats(Ivol8c_roi[i], fatseg1vol_mask_R3_ken[i], fatseg2_refinedvol_mask_R3_ken[i],3)
#         axs[3].set_title("WITH Z connections")
#         axs[4].imshow(final_fat_ken[i], cmap='bone')
#         axs[4].set_title("WITH Z connections")
#     all_data_table=calc2(fatseg1vol_R2_ken,fatseg2_refinedvol_R3_ken,final_fat_ken)
#     print ("------------------------------------------------------------------")
#     #get file # in final total volume!!
    

## TESTS

#### Threshold 1 + 2 Separated

In [ ]:
# #test area calculation between mask vs no mask 
# print(np.sum(roimaskvol[7]>0))
# print(np.sum(roivol[7]>0))

# fig, axs = plt.subplots (1, 2, figsize=(15,10))
# axs[0].imshow(roimaskvol[7],cmap="bone")
# axs[1].imshow(roivol[7],cmap="bone")

In [ ]:

# muscleroi=muscleroi2=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

# def calc_vol(fat):
#     im_spacing = image.GetSpacing() #get spacing for original image - for calculations later
#     for i in range(j):
#         muscleroi[i]=roimaskvol[i]-fat[i] 
#         #MUSCLE AND FAT AREA
#         MuscFatAreaPix=np.sum(roimaskvol[i]>0) #same result as roivol>0 #not musclemask>0 cus musclemask INCLUDES bone area! roimaskvol is ONLY muscle+fat. #Also don’t just do roi, need roi>0
#         MuscFatAreaMM=MuscFatAreaPix*im_spacing[0]*im_spacing[1] #multiply by x and y to convert to mm:  1 pixel here= spacing 0.9765625,0.9765625,5.0
#         #FAT AREA
#         FatAreaPix=np.sum(fat[i]>0) 
#         FatAreaMM=FatAreaPix*im_spacing[0]*im_spacing[1]
#         #MUSCLE AREA
#         MuscAreaPix=np.sum(muscleroi[i]>0)
#         MuscAreaMM=MuscAreaPix*im_spacing[0]*im_spacing[1]
#         #FAT PERCENTAGE
#         FatPerc=FatAreaPix*100/MuscFatAreaPix #adding 2 dashes rounds it DOWN to whole number
#         #VOLUME OF EACH SLICE
#         MuscFatVolMM=MuscFatAreaMM*im_spacing[2]
#         FatVolMM=FatAreaMM*im_spacing[2] #multiply by z -slice thickness
#         MuscVolMM=MuscAreaMM*im_spacing[2]
#         print(f"\nSlice {i+1}\n")
#         print(tabulate([['Musc + Fat Area', "%.1f" % MuscFatAreaPix,'pixels'], ['Musc + Fat Area',"%.1f" % MuscFatAreaMM,'mm^2'],
#                 ['Fat Area', "%.1f" % FatAreaPix,'pixels'],  ['Fat Area', "%.1f" % FatAreaMM,'mm^2'], ['Musc Area',"%.1f" % MuscAreaPix,'pixels'],['Musc Area',"%.1f" % MuscAreaMM,'mm^2'],
#                 ['Fat Perc', "%.1f" % FatPerc,'%'],['Musc + Fat Vol', "%.1f" % MuscFatVolMM,'mm^3'],['Fat Vol', "%.1f" % FatVolMM,'mm^3'],['Musc Vol', "%.1f" % MuscVolMM,'mm^3']],
#                 headers=['Data', 'Value','Units']))
#         print ("------------------------------------------------------------")

# #calc_vol(fatseg1vol_R1) # R1 threshold 1 fat - no 3D check
# #calc_vol(fatsegfinalvol_R1) #R1 threshold 2 fat - no 3D check
# #calc_vol(final_fat) #R2 threshold 2 fat WITH final 3D check (final fat from R2)

#### Threshold 1 + 2 Combined + Fat Correction

In [ ]:

# muscleroi=muscleroi2=np.empty([Ivol8.shape[0], Ivol8.shape[1], Ivol8.shape[2]], dtype='uint8')

# #problem - printing float separately works (354.0) but not in table
# def calc(fat1,fat2,fat3):
#     im_spacing = image.GetSpacing() #get spacing for original image - for calculations later
#     for i in range(j):
#     #MUSCLE + FAT AREA
#         MuscFatAreaPix=np.sum(roivol[i]>0)   
#         MuscFatAreaMM=MuscFatAreaPix*im_spacing[0]*im_spacing[1] 
#         MuscFatVolMM=MuscFatAreaMM*im_spacing[2]
#     #THRESHOLD 1 FAT
#         muscleroi1=roivol[i]-fat1[i] 
#         #plt.imshow(muscleroi1,cmap="bone")
#         #plt.imshow(roivol[i],cmap="bone")
#         #FAT AREA
#         FatAreaPix1=np.sum(fat1[i]>0) 
#         FatAreaMM1=FatAreaPix1*im_spacing[0]*im_spacing[1]
#         #MUSCLE AREA
#         MuscAreaPix1=np.sum(muscleroi1>0)
#         MuscAreaMM1=MuscAreaPix1*im_spacing[0]*im_spacing[1]
#         #FAT PERCENTAGE
#         FatPerc1=FatAreaPix1*100/MuscFatAreaPix #adding 2 dashes rounds it DOWN to whole number
#         #VOLUME OF EACH SLICE
#         FatVolMM1=FatAreaMM1*im_spacing[2] #multiply by z -slice thickness
#         MuscVolMM1=MuscAreaMM1*im_spacing[2]
#         print(f"\nSlice {i+1} Threshold #1 \n")
    
#         print(tabulate([['Musc + Fat Area', "%.1f" % MuscFatAreaPix,'pixels'], ['Musc + Fat Area',"%.1f" % MuscFatAreaMM,'mm^2'],['Musc + Fat Vol', "%.1f" % MuscFatVolMM,'mm^3'],[],
#                 ['Musc Area 1',"%.1f" % MuscAreaPix1,'pixels'],['Musc Area 1',"%.1f" % MuscAreaMM1,'mm^2'],['Musc Vol 1', "%.1f" % MuscVolMM1,'mm^3'],[],
#                 ['Fat Area 1', "%.1f" % FatAreaPix1,'pixels'],  ['Fat Area 1', "%.1f" % FatAreaMM1,'mm^2'], 
#                 ['Fat Perc 1', "%.1f" % FatPerc1,'%'],['Fat Vol 1', "%.1f" % FatVolMM1,'mm^3']],
#                 headers=['Data', 'Value','Units']))
#         print (f"\n")
        
#     #THRESHOLD 2 FAT
#         #NOT including threshold 1 fat----------------------------------------------
#         muscleroi2=roivol[i]-fat2[i] 
#         #FAT AREA
#         FatAreaPix2=np.sum(fat2[i]>0)
#         FatAreaMM2=FatAreaPix2*im_spacing[0]*im_spacing[1]
#         #MUSCLE AREA
#         MuscAreaPix2=np.sum(muscleroi2>0)
#         MuscAreaMM2=MuscAreaPix2*im_spacing[0]*im_spacing[1]
#         #FAT PERCENTAGE
#         FatPerc2=FatAreaPix2*100/MuscFatAreaPix
#         #VOLUME OF EACH SLICE
#         FatVolMM2=FatAreaMM2*im_spacing[2]
#         MuscVolMM2=MuscAreaMM2*im_spacing[2]
        
      
#         #FAT CORRECTION 
#         Fat1masked=np.ma.masked_where(fat1[i]==0,fat1[i]) #brighter fat
#         Fat2masked=np.ma.masked_where(fat2[i]==0,fat2[i]) #less bright fat
#         FatSegI_1=np.mean(Fat1masked) 
#         FatSegI_2=np.mean(Fat2masked)
#         cfactor=(FatSegI_2/FatSegI_1) #fat correction factor
        
        
#         FatVolCombined=FatVolMM1+FatVolMM2 #final volume not corrected
#         FatVolMM2_C=FatVolMM2*cfactor 
#         FatVolCombined_C=FatVolMM1+FatVolMM2_C #final volume corrected
#         FatPerc2_C=FatPerc2*cfactor
        
#         print(f"\nSlice {i+1} Threshold #2 NOT including fat 1  \n")
#         print(tabulate([['Musc Area 2',"%.1f" % MuscAreaPix2,'pixels'],['Musc Area 2',"%.1f" % MuscAreaMM2,'mm^2'],['Musc Vol 2', "%.1f" % MuscVolMM2,'mm^3'],[],
#                 ['Fat Area 2', "%.1f" % FatAreaPix2,'pixels'],['Fat Area 2 ', "%.1f" % FatAreaMM2,'mm^2'],
#                 ['Fat Perc 2', "%.1f" % FatPerc2,'%'],['Fat Perc 2 Corrected', FatPerc2_C,'%'],['Fat Vol 2 Not Corrected', "%.1f" % FatVolMM2,'mm^3'],['Fat Vol 2 Corrected', "%.1f" % FatVolMM2_C,'mm^3']],
#                 headers=['Data', 'Value','Units']))
#         print (f"\n")
#         print(f"\nSlice {i+1} Fat Correction \n")
#         print(tabulate([['Fat 1 Intensity', FatSegI_1,'pixels'],['Fat 2 Intensity', FatSegI_2,'mm^2'],['Fat Correction Factor', cfactor],[],],
#                 headers=['Data', 'Value','Units']))
#         print (f"\n")
        
    
#     #FINAL FAT using Threshold 2 ----------------------------------------------------
#         muscleroi3=roivol[i]-fat3[i]
#         #FAT AREA
#         FatAreaPix3=np.sum(fat3[i]>0)
#         FatAreaMM3=FatAreaPix3*im_spacing[0]*im_spacing[1]
#         #MUSCLE AREA
#         MuscAreaPix3=np.sum(muscleroi3>0)
#         MuscAreaMM3=MuscAreaPix3*im_spacing[0]*im_spacing[1]
#         #FAT PERCENTAGE
#         FatPerc3=FatAreaPix3*100/MuscFatAreaPix
#         #VOLUME OF EACH SLICE
#         FatVolMM3=FatAreaMM3*im_spacing[2] #this is same as FatVolCombined NOT corrected- use this as a check
#         MuscVolMM3=MuscAreaMM3*im_spacing[2]
        
        
#         print(f"\nSlice {i+1} Threshold #2 INCLUDING fat 1 \n")
#         print(tabulate([['Musc Area 3',"%.1f" % MuscAreaPix3,'pixels'],['Musc Area 3',"%.1f" % MuscAreaMM3,'mm^2'],['Musc Vol 3', "%.1f" % MuscVolMM3,'mm^3'],[],
#                 ['Fat Area 3', "%.1f" % FatAreaPix3,'pixels'],['Fat Area 3 ', "%.1f" % FatAreaMM3,'mm^2'],
#                 ['Fat Perc 3', "%.1f" % FatPerc3,'%'],['Fat Vol 3', "%.1f" % FatVolMM3,'mm^3'],[],
#                 ['Fat Volume Combined Not Corrected', "%.1f" % FatVolCombined,'MM^3'],['Fat Volume Combined Corrected', "%.1f" % FatVolCombined_C,'MM^3']],
#                 headers=['Data', 'Value','Units']))
    
        
#         print ("------------------------------------------------------------")
    
    
# # calc(fatseg1vol_R1,fatseg2_refinedvol,fatsegfinalvol_R1)
# calc(fatseg1vol_R2_ken,fatseg2_refinedvol_R3_ken,final_fat_ken)



In [ ]:
# #Checking for one slice 
# def fat_correction(i):
#     im_spacing = image.GetSpacing() 
#     fat1masked=np.ma.masked_where(fatseg1vol_R1[i]==0,fatseg1vol_R1[i])
#     fat2masked=np.ma.masked_where(fatseg2_refinedvol[i]==0,fatseg2_refinedvol[i])
#     FatSegI_1=np.mean(fat1masked) 
#     FatSegI_2=np.mean(fat2masked)
#     cfactor=(FatSegI_2/FatSegI_1)

#     #FAT AREA
#     FatAreaPix1=np.sum(fatseg1vol_R1[i]>0)
#     FatAreaMM1=FatAreaPix1*im_spacing[0]*im_spacing[1]
#     FatVolMM1=FatAreaMM1*im_spacing[2]
    
#     FatAreaPix2=np.sum(fatseg2_refinedvol[i]>0)
#     FatAreaMM2=FatAreaPix2*im_spacing[0]*im_spacing[1]
#     FatVolMM2=FatAreaMM2*im_spacing[2]
#     FatVolMM2_C=FatAreaMM2*im_spacing[2]*cfactor
    
#     FatAreaPix3=np.sum(fatsegfinalvol_R1[i]>0)
#     FatAreaMM3=FatAreaPix3*im_spacing[0]*im_spacing[1]

#     #VOLUME OF EACH SLICE
#     FatVolMM3=FatAreaMM3*im_spacing[2] #this is same as FatVolCombined NOT corrected
#     FatVolMM12_notC=FatVolMM1+FatVolMM2 
#     FatVolMM3_C=FatVolMM1+FatVolMM2_C #should be same as FatVolMM12_notC

#     print (f"Fat 1 intensity = {np.mean(fat1masked)}\nFat 2 intensity={np.mean(fat2masked)}\nFat Correction Factor={cfactor}\nFat3 volume NOT corrected={FatVolMM3}\nFat1+2 NOT corrected={FatVolMM12_notC}\nFat corrected={FatVolMM3_C}")

# fat_correction(7)

In [ ]:
#TEST for final product
#compare Z-connection check vs no check (R3 vs R1 fat)
#why does fats need to be IN the function for it to work?
# def NoZ_vs_Z(): 
    
#     for i in range(j): 
#         fig, axs = plt.subplots (1, 5, figsize=(17,55))

#         axs[0].imshow(Ivol8c_roi[i], cmap='bone')
#         axs[0].set_title(f"Slice {i+1} Ivol8c_roi")
#         fats(Ivol8c_roi[i], fatseg1vol_mask_R1[i], fatseg2_refinedvol_mask[i],1)
#         axs[1].set_title("WITHOUT Z connections")
#         axs[2].imshow(fatsegfinalvol_R1[i], cmap='bone')
#         axs[2].set_title("WITHOUT Z connections")

#         fats(Ivol8c_roi[i], fatseg1vol_mask_R3_ken[i], fatseg2_refinedvol_mask_R3_ken[i],3)
#         axs[3].set_title("WITH Z connections")
#         axs[4].imshow(final_fat_ken[i], cmap='bone')
#         axs[4].set_title("WITH Z connections")

# NoZ_vs_Z()